In [180]:
import pandas as pd
import folium
import numpy as np
from math import radians, cos, sin, asin, sqrt
from ipyleaflet import Map, Marker, Popup, Icon
import ipywidgets as widgets
from IPython.display import display
from shapely.geometry import Point, Polygon
from sqlalchemy import create_engine, text # needed for DB connection
import json

<h1>Speed Flows<h1>

In [91]:
file_path = 'C:/Users/migue/source/repos/DataScience/datasets/speed_flow/march/intensiteit-snelheid-export 1 maart_metadata.zip'
# Define the columns to load
cols_to_load = ['id_meetlocatie', 'start_locatie_longitude', 'start_locatie_latitude', 'wegnaam_vild', 'rijstrook_rijbaan']
cols_for_uniqueness = ['id_meetlocatie', 'start_locatie_longitude', 'start_locatie_latitude']
# Initialize an empty DataFrame to hold unique rows
unique_rows = pd.DataFrame(columns=cols_to_load)

# Set chunk size
chunk_size = 20000

# Read the CSV file in chunks
for chunk in pd.read_csv(file_path, usecols=cols_to_load, chunksize=chunk_size):
    # Drop duplicates within the chunk
    chunk_unique = chunk.drop_duplicates(subset=cols_to_load)
    
    # Concatenate the unique rows from this chunk with the main DataFrame, and drop duplicates again
    unique_rows = pd.concat([unique_rows, chunk_unique]).drop_duplicates(subset=cols_for_uniqueness)

# Print the size of the reduced DataFrame
print(unique_rows.shape)
unique_rows.head()

C:\Users\migue\AppData\Local\Temp\ipykernel_29764\695956152.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  unique_rows = pd.concat([unique_rows, chunk_unique]).drop_duplicates(subset=cols_for_uniqueness)
C:\Users\migue\AppData\Local\Temp\ipykernel_29764\695956152.py:12: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, usecols=cols_to_load, chunksize=chunk_size):
C:\Users\migue\AppData\Local\Temp\ipykernel_29764\695956152.py:12: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, usecols=cols_to_load, chunksize=chunk_size):
C:\Users\migue\AppData\Local\Temp\ipyk

(1452, 5)


,id_meetlocatie,start_locatie_longitude,start_locatie_latitude,wegnaam_vild,rijstrook_rijbaan
0,GEO0B_R_RWSTI356574,4.36867,51.91099,Ring Rotterdam,lane1
17280,GEO0B_R_RWSTI357017,4.33028,51.91749,NaN,lane1
34560,GEO0B_R_RWSTI357030,4.33322,51.91892,NaN,lane1
43200,GEO0B_R_RWSTI357031,4.33499,51.91891,NaN,lane1
51840,GEO0B_R_RWSTI357032,4.32780,51.91782,NaN,lane1


Filter speed flows for region, direction, and lanes

In [92]:
# filter so that only rows where wegnaam_vild equals 'Rotterdam Ring' are included
unique_rows = unique_rows[unique_rows['wegnaam_vild'] == 'Ring Rotterdam']
print('Rows:', len(unique_rows))

# Convert coordinates to floats
unique_rows['start_locatie_longitude'] = pd.to_numeric(unique_rows['start_locatie_longitude'], errors='coerce')
unique_rows['start_locatie_latitude'] = pd.to_numeric(unique_rows['start_locatie_latitude'], errors='coerce')

# Drop rows with NaN values in 'primaire_locatie_lengtegraad' or 'primaire_locatie_breedtegraad'
unique_rows.dropna(subset=['start_locatie_longitude', 'start_locatie_latitude'], inplace=True)

unique_rows = unique_rows.drop_duplicates(subset=['start_locatie_longitude', 'start_locatie_latitude'], keep='first')

min_longitude = unique_rows['start_locatie_longitude'].min()
max_longitude = unique_rows['start_locatie_longitude'].max()
min_latitude = unique_rows['start_locatie_latitude'].min()
max_latitude = unique_rows['start_locatie_latitude'].max()

# Truncate to 6 decimal places without rounding using string formatting
min_latitude = float(format(min_latitude))
max_latitude = float(format(max_latitude))
min_longitude = float(format(min_longitude))
max_longitude = float(format(max_longitude))

print(max_latitude)
print(min_latitude)
print(max_longitude)
print(min_longitude)
unique_rows.head()

#List of points to be excluded for direction 1, converted to tuples
excluded_points_direction_1 = [(51.9236, 4.35371), (51.92458, 4.35778),(51.92464, 4.36257), (51.92556, 4.36336), 
    (51.92644, 4.36252), (51.92564, 4.36606), (51.92428, 4.36593), (51.92415, 4.36559), (51.92309, 4.3646), 
    (51.92668, 4.36557), (51.92377, 4.36762), (51.924609, 4.36925), (51.92715, 4.36751), (51.927457, 4.367002), 
    (51.92607, 4.36988), (51.92575, 4.37079), (51.93292, 4.36435),  (51.926146, 4.37316),  (51.92591, 4.37302),
    (51.92579, 4.37318),  (51.926116, 4.374173), (51.925483, 4.373974), (51.925515, 4.374958),(51.93292, 4.36435), 
    (51.926146, 4.37316), (51.92591, 4.37302), (51.92579, 4.37318), (51.926116, 4.374173), (51.925483, 4.373974), 
    (51.925515, 4.374958),(51.92732, 4.39672), (51.92714, 4.39668), (51.92683, 4.40015), (51.926754, 4.402966), 
    (51.9259, 4.40215), (51.925861, 4.402185), (51.925319, 4.40346), (51.92556, 4.40528), (51.9253, 4.40588), 
    (51.92385, 4.40907), (51.92279, 4.4144), (51.92266, 4.41453), (51.92281, 4.41666), (51.92392, 4.42017), 
    (51.92388, 4.4203), (51.92525, 4.4242), (51.92542, 4.42407), (51.92684, 4.42757), (51.92847, 4.43154), 
    (51.92927, 4.43347), (51.92968, 4.43534), (51.93063, 4.43763), (51.93118, 4.43959), (51.93167, 4.43996), 
    (51.93504, 4.4347), (51.92614, 4.37808), (51.92602, 4.37816), (51.92632, 4.38242), (51.92664, 4.38726), 
    (51.92688, 4.39087), (51.92636, 4.40261), (51.933556, 4.443744), (51.93379, 4.44482), (51.93431, 4.44585), (51.93489, 4.44902),
    (51.93602, 4.45427), (51.93753, 4.45796), (51.93747, 4.45828), (51.93916, 4.46289), (51.93902, 4.46354), (51.93986, 4.46565), 
    (51.94046, 4.46989), (51.94133, 4.47685), (51.94182, 4.48142), (51.941664, 4.482457), (51.9429, 4.48992), (51.94483, 4.49481),
    (51.94519, 4.49705), (51.94594, 4.50408), (51.94679, 4.51015), (51.94714, 4.51226), (51.94784, 4.51593),  (51.94906, 4.52115),  
    (51.94938, 4.52499),  (51.95036, 4.52719), (51.950725, 4.526611), (51.950771, 4.526861), (51.95133, 4.53284),  (51.95105, 4.5345),
    (51.95188, 4.53681), (51.94988, 4.53203), (51.94796, 4.53206),   (51.9471, 4.533383),   (51.9468, 4.53079),    (51.94579, 4.53253),
    (51.93059, 4.53852), (51.92731, 4.5391), (51.92603, 4.53618), (51.92659, 4.53883), (51.92385, 4.53787),    (51.92226, 4.53657),    (51.91979, 4.53522),    (51.91806, 4.53487),
    (51.91804683, 4.53500571),    (51.91754, 4.53368),    (51.91543, 4.53544),    (51.911633, 4.537962),
    (51.91128, 4.53799),    (51.9115, 4.53695),    (51.91128, 4.53799),    (51.90882, 4.53971),
    (51.90636, 4.54145),    (51.90224, 4.54436),    (51.89848, 4.54813), (51.896641, 4.54674),  (51.8956, 4.55554),   (51.89319, 4.55986),  (51.894112, 4.560175),
    (51.89413, 4.560257), (51.88926, 4.56267),  (51.885892, 4.563197), (51.88511, 4.56438), (51.88193, 4.56621),  (51.8785, 4.56813),   (51.87691, 4.56889),   (51.87679, 4.56917),
    (51.87516, 4.57733), (51.896674, 4.546752), (51.87888, 4.56548),  (51.87549, 4.56944),  (51.87515, 4.56978),
    (51.87463,  4.5701),   (51.87348, 4.57446), (51.8735,  4.57158),  (51.87386, 4.57022), (51.91965, 4.3661),   (51.9174, 4.36655),  (51.91524, 4.36723),  (51.91398, 4.36839),
    (51.9127, 4.36821),   (51.91267, 4.36845), (51.91085, 4.36947),  (51.91087, 4.36916),
    (51.91099, 4.36867),  (51.910187, 4.370783), (51.90839, 4.37014), (51.90838, 4.37042),
    (51.90562, 4.37024),  (51.90561, 4.37005), (51.90452, 4.37004),  (51.9033, 4.36989),
    (51.90329, 4.3697),   (51.9019, 4.36999),  (51.90188, 4.36981),  (51.90061, 4.37029),
    (51.9006, 4.37012),   (51.89953, 4.37079), (51.89951, 4.37061),  (51.89825, 4.37138),
    (51.89829, 4.37159),  (51.89727, 4.37245), (51.89723, 4.37224),  (51.89628, 4.3733),
    (51.89632, 4.3735),   (51.89534, 4.37443), (51.89538, 4.37462),  (51.89429, 4.37573),
    (51.89391, 4.37633),  (51.89224, 4.37763), (51.89202, 4.37749),  (51.89007, 4.37807),
    (51.89013, 4.37837),  (51.88537, 4.38041), (51.88134, 4.38281),  (51.87862, 4.38409),
    (51.87693, 4.38418),  (51.877, 4.38481),   (51.87546, 4.38273),  (51.87513, 4.37946),
    (51.87413, 4.37828),  (51.87363, 4.3781),  (51.87364, 4.3789),   (51.90452, 4.36983), (51.87696, 4.38409), (51.87213, 4.38437), 
    (51.87324, 4.38312), (51.87388, 4.38593), (51.87285, 4.38364), (51.87306, 4.39103), (51.87317, 4.39703), (51.87322, 4.39814),
    (51.87321, 4.40125), (51.87271, 4.41508), (51.87197, 4.42052), (51.8689, 4.43428),
    (51.86608, 4.44719), (51.86523, 4.45057), (51.86345, 4.45638), (51.8615, 4.46537),
    (51.86159, 4.47045), (51.86188, 4.47651), (51.86235, 4.48123), (51.86332, 4.48787),
    (51.86406, 4.49301), (51.86448, 4.49598), (51.86481, 4.50021), (51.86529, 4.50362),
    (51.86507, 4.50685), (51.86516, 4.5106), (51.86412, 4.51339), (51.86399, 4.51343),
    (51.92444, 4.35799),  (51.87228, 4.39258),  (51.87187, 4.39189),  (51.87356, 4.40533),
    (51.87328, 4.40779),  (51.87321, 4.41174),  (51.87222, 4.42038),  (51.87073, 4.42772),
    (51.86894, 4.43446),  (51.86909, 4.43595),  (51.86782, 4.44053),  (51.86694, 4.44486),
    (51.86655, 4.44569),  (51.86539, 4.45073),  (51.86187, 4.47053),  (51.86199, 4.47659),
    (51.86353, 4.48779),  (51.86409, 4.49160),  (51.86460, 4.49769),  (51.86527, 4.51360),
    (51.86417, 4.52640),  (51.86370, 4.53036),  (51.86352, 4.53462),  (51.92444, 4.35799), 
    (51.87228, 4.39258), (51.87187, 4.39189), (51.87356, 4.40533),
    (51.87328, 4.40779), (51.87321, 4.41174), (51.87222, 4.42038), (51.87073, 4.42772),
    (51.86894, 4.43446), (51.86909, 4.43595), (51.86782, 4.44053), (51.86694, 4.44486),
    (51.86655, 4.44569), (51.86539, 4.45073), (51.86187, 4.47053), (51.86199, 4.47659),
    (51.86353, 4.48779), (51.86409, 4.4916),  (51.8646,  4.49769), (51.86527, 4.5136),
    (51.86417, 4.5264),  (51.8637,  4.53036), (51.86352, 4.53462), (51.86385, 4.45669),  
    (51.86242, 4.46352),  (51.86210, 4.47649),  (51.86257, 4.48116),
    (51.86480, 4.49762),  (51.86419, 4.51512),  (51.86459, 4.51747),  (51.86515, 4.51878),
    (51.86477, 4.52089),  (51.86515, 4.51878),  (51.86477, 4.52089),  (51.86467, 4.52280),
    (51.86371, 4.53855),  (51.86430, 4.54267),  (51.86539, 4.54786),  (51.86613, 4.55061),
    (51.86613, 4.55061),  (51.85979, 4.51599),  (51.86385, 4.45669),  (51.86242, 4.46352),  (51.86210, 4.47649),  (51.86257, 4.48116),
    (51.86480, 4.49762),  (51.86419, 4.51512),  (51.86459, 4.51747),  (51.86515, 4.51878),
    (51.86477, 4.52089),  (51.86477, 4.52089),  (51.86467, 4.52280),  (51.86371, 4.53855),
    (51.86430, 4.54267),  (51.86539, 4.54786),  (51.86613, 4.55061),  (51.86613, 4.55061),
    (51.85979, 4.51599), (51.87334, 4.39603),  (51.87346, 4.3966),   (51.87333, 4.39049),  (51.87353, 4.40137),
    (51.87082, 4.4281),   (51.86794, 4.44074),  (51.8655, 4.51333),   (51.86583, 4.51823),
    (51.86523, 4.52332),  (51.86537, 4.52334),  (51.86462, 4.52828),  (51.86406, 4.53391),
    (51.86584, 4.54843),  (51.86773, 4.55059),  (51.86752, 4.55366),  (51.86773, 4.55356),
    (51.86866, 4.55689),  (51.86896, 4.55712),  (51.86909, 4.55705),  (51.8695, 4.56073),
    (51.87054, 4.56078),  (51.87052, 4.56433),  (51.87156, 4.56442),  (51.8719, 4.56713),
    (51.87215, 4.56711),  (51.87223, 4.5658),   (51.871135, 4.572183), (51.87146, 4.57264),
    (51.87264, 4.57666),  (51.86695, 4.51762),  (51.86711, 4.54985),  (51.86681, 4.55065),  (51.87344, 4.56802),
    (51.87120, 4.57389),  (51.87064, 4.57338),  (51.87064, 4.57338),  (51.86892, 4.57671),
    (51.86816, 4.57899),  (51.86842, 4.57869),  (51.86866, 4.57927),  (51.85837, 4.59255), (51.86583, 4.58378), (51.86324, 4.58827), 
    (51.86104, 4.5911),   (51.86963, 4.51556),  (51.87367, 4.56829),  
]

excluded_points_same_direction = [
    (51.91093, 4.36979), (51.91126, 4.37013), (51.90797, 4.37085), (51.90512, 4.3709),
    (51.90327, 4.37071), (51.90199, 4.37081), (51.90073, 4.37112), (51.89967, 4.37161),
    (51.89844, 4.37241), (51.89744, 4.37327), (51.89647, 4.37431), (51.89554, 4.37542),
    (51.89402, 4.37697), (51.8923, 4.37803), (51.89009, 4.3787),   (51.87401, 4.39544), 
    (51.87235, 4.42059), (51.87092, 4.42812),

]

excluded_points_same_direction_lower_part_ring = [
    (51.87359, 4.39646), (51.87287, 4.41689), (51.87344, 4.41094), (51.87364, 4.40742),
    (51.87374, 4.40381), (51.87359, 4.39646), (51.87319, 4.39058), (51.87391, 4.39601),
    (51.87211, 4.42385), (51.87127, 4.42663), (51.86984, 4.43353), (51.86655, 4.4471),
    (51.86468, 4.45405), (51.8627, 4.46276), (51.86203, 4.47039), (51.86209, 4.47362),
    (51.8627, 4.48069), (51.86366, 4.48744), (51.86413, 4.49061), (51.86493, 4.497),
    (51.86545, 4.50346), (51.8657, 4.50771), (51.86554, 4.50801), (51.86573, 4.51184),
    (51.86536, 4.51853), (51.86427, 4.52714), (51.86435, 4.52795), (51.86377, 4.53365),
    (51.86405, 4.53912), (51.86776, 4.55296), (51.8694, 4.55711), (51.86928, 4.43659), 
    (51.86817, 4.44046), (51.86525, 4.45295), (51.86216, 4.47041), (51.86377, 4.48724), 
    (51.8721, 4.56288), (51.87375, 4.56613),

]

total_excluded_points = excluded_points_direction_1 + excluded_points_same_direction + excluded_points_same_direction_lower_part_ring

# Mask to exclude these points
mask_unique = unique_rows.apply(
    lambda row: any(
        abs(row['start_locatie_latitude'] - point[0]) < 1e-6 and 
        abs(row['start_locatie_longitude'] - point[1]) < 1e-6 
        for point in total_excluded_points
    ), axis=1
)

# Apply mask to filter out rows
unique_rows = unique_rows[~mask_unique]
unique_rows.reset_index(drop=True, inplace=True)

# Print results
print(f'Rows after filtering: {len(unique_rows)}')

Rows: 634
51.95385
51.85837
4.59255
4.35371
Rows after filtering: 109


Add a Segment column to classify the cameras by section of the road for a better mapping to incidents

In [93]:
# Define the corner points of the rectangle
north_west = (51.925170, 4.361240)
north_east = (51.953850, 4.552810) 
south_east = (51.874718, 4.565618)
south_west = (51.875290, 4.388910)

def label_segments_lat_long_band(df, north_west, north_east, south_east, south_west):

    # Lists of IDs for special classification
    west_east_ids = [
        'RWS01_MONIBAS_0201hrr0264ra', 
        'RWS01_MONIBAS_0201hrr0266ra', 
        'RWS01_MONIBAS_0201hrr0269ra', 
        'RWS01_MONIBAS_0201hrr0272ra', 
        'RWS01_MONIBAS_0201hrr0274ra'
    ]
    south_ids = [
        'RWS01_MONICA_10D01002C80DB820000F', 
        'GEO0B_R_RWSTI357698', 
        'RWS01_MONIBAS_0160vwx0182ra',
        'RWS01_MONICA_10D01002C80DB8200007'
    ]
    
    # Initialize the segment column
    df['segment'] = 'other'  # default value
    
    
    # Check each point against the latitude and longitude bands
    for i, row in df.iterrows():
        lat = row['start_locatie_latitude']
        long = row['start_locatie_longitude']
        id_meetlocatie = row['id_meetlocatie']
        
        if id_meetlocatie in west_east_ids:
            df.at[i, 'segment'] = 'west-east'
        elif id_meetlocatie in south_ids:
            df.at[i, 'segment'] = 'south'

        elif north_west[0] <= lat <= north_east[0]:
            df.at[i, 'segment'] = 'west-east'
        elif south_west[0] <= lat <= north_west[0] and north_west[1] <= long<= south_west[1]:
            df.at[i, 'segment'] = 'north'
        elif south_east[0] <= lat <= north_east[0] and 4.525670 <= long <= south_east[1]:
            df.at[i, 'segment'] = 'south'
        elif 51.923790 <= lat <= 51.924990  and 4.420440 <= long <= 4.406130:
            df.at[i, 'segment'] = 'west-east'
        elif min_latitude <= south_west[0]:
            df.at[i, 'segment'] = 'east-west'
    
    return df

# Apply the labeling function
unique_rows = label_segments_lat_long_band(unique_rows, north_west, north_east, south_east, south_west)
unique_rows.head()


,id_meetlocatie,start_locatie_longitude,start_locatie_latitude,wegnaam_vild,rijstrook_rijbaan,segment
0,GEO0B_R_RWSTI357698,4.537880,51.927530,Ring Rotterdam,lane1,south
1,GEO0K_K_RWSTI357132,4.366503,51.922442,Ring Rotterdam,lane1,north
2,GEO0K_K_RWSTI357777,4.565618,51.874718,Ring Rotterdam,lane1,south
3,RWS01_MONIBAS_0040vwe0763ra,4.384620,51.879240,Ring Rotterdam,lane1,north
4,RWS01_MONIBAS_0040vwn0713ra,4.368560,51.914020,Ring Rotterdam,lane1,north


Map generation of the cameras

In [94]:
# Function to determine marker color
def color_for_segment(segment):
    if segment == 'west-east':
        return 'red'
    elif segment == 'south':
        return 'blue'
    elif segment == 'east-west':
        return 'green'
    elif segment == 'north':
        return 'orange'
    else:
        return 'gray'  # In case the segment is undefined

# Create a map centered on the mean latitude and longitude
m = folium.Map(location=[unique_rows['start_locatie_latitude'].mean(), unique_rows['start_locatie_longitude'].mean()], zoom_start=10)

# Add a marker for each unique meetlocatie
for i, row in unique_rows.iterrows():
    folium.Marker(
        location=[row['start_locatie_latitude'], row['start_locatie_longitude']],
        icon=folium.Icon(color=color_for_segment(row['segment'])),
        popup=(
            f"ID: {row['id_meetlocatie']}<br>"
            f"Latitude: {row['start_locatie_latitude']:.6f}<br>"
            f"Longitude: {row['start_locatie_longitude']:.6f}"
        ),
    ).add_to(m)

# Show the map
m.save('unique_meetlocaties.html')
m

<h1>Incidents</h1>

In [169]:
df7 = pd.read_csv('C:/Users/migue/source/repos/DataScience/datasets/status/sb-incidenten-export_rotterdam 1 maart - 31 augustus.zip', compression='zip')
print(df7.columns)
df7.head()

Index(['id', 'versie', 'type', 'bron', 'starttijd', 'eindtijd',
       'situatie_aangemaakt_tijd', 'situatie_versie_tijd', 'speciaal_geval',
       'vild_versie', 'vild_richting', 'vild_primaire_locatie',
       'vild_afstand_tot_primaire_locatie', 'vild_secundaire_locatie',
       'vild_afstand_tot_secundaire_locatie', 'mobiliteit',
       'veiligheids_gerelateerd_bericht', 'primaire_locatie_lengtegraad',
       'primaire_locatie_breedtegraad', 'secundaire_locatie_lengtegraad',
       'secundaire_locatie_breedtegraad', 'gedetailleerd_type'],
      dtype='object')


C:\Users\migue\AppData\Local\Temp\ipykernel_29764\533038565.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df7 = pd.read_csv('C:/Users/migue/source/repos/DataScience/datasets/status/sb-incidenten-export_rotterdam 1 maart - 31 augustus.zip', compression='zip')


,id,versie,type,bron,starttijd,eindtijd,situatie_aangemaakt_tijd,situatie_versie_tijd,speciaal_geval,vild_versie,...,vild_afstand_tot_primaire_locatie,vild_secundaire_locatie,vild_afstand_tot_secundaire_locatie,mobiliteit,veiligheids_gerelateerd_bericht,primaire_locatie_lengtegraad,primaire_locatie_breedtegraad,secundaire_locatie_lengtegraad,secundaire_locatie_breedtegraad,gedetailleerd_type
0,A-ALL-IM19087680NLD_1,2,vehicle_obstruction,ANWB,2019-08-28 11:34:31,2020-09-09 14:32:53,2019-08-28 12:11:32,2019-08-28 12:11:32,NaN,5.17A,...,NaN,9200.0,NaN,stationary,True,4.346407,52.04192,4.346407,52.04192,brokenDownVehicle
1,A-ALL-IM19087680NLD_1,3,vehicle_obstruction,ANWB,2019-08-28 11:34:31,2020-09-09 14:32:53,2019-09-11 11:49:12,2019-09-11 11:49:12,NaN,5.18A,...,NaN,9200.0,NaN,stationary,True,4.346407,52.04192,4.346407,52.04192,brokenDownVehicle
2,A-ALL-IM19087680NLD_1,4,vehicle_obstruction,ANWB,2019-08-28 11:34:31,2020-09-09 14:32:53,2019-12-11 11:32:28,2019-12-11 11:32:28,NaN,5.19A,...,4100.0,9197.0,100.0,stationary,True,4.346407,52.04192,NaN,NaN,brokenDownVehicle
3,A-ALL-IM19087680NLD_1,5,vehicle_obstruction,ANWB,2019-08-28 11:34:31,2019-12-16 18:45:27,2019-12-11 11:32:28,2019-12-11 11:32:28,snapshot_closed,5.19A,...,4100.0,9197.0,100.0,stationary,True,4.346407,52.04192,NaN,NaN,brokenDownVehicle
4,A-ALL-IM19087680NLD_1,6,vehicle_obstruction,ANWB,2019-08-28 11:34:31,2020-09-09 14:32:53,2020-06-24 12:40:00,2020-06-24 12:40:00,NaN,5.21A,...,4100.0,9197.0,100.0,stationary,True,4.346407,52.04192,NaN,NaN,brokenDownVehicle


In [170]:
incidents_df = df7[['id', 'starttijd', 'eindtijd', 'type', 'gedetailleerd_type', 'primaire_locatie_lengtegraad', 'primaire_locatie_breedtegraad']]
print(len(incidents_df))
incidents_df.head()

161962


,id,starttijd,eindtijd,type,gedetailleerd_type,primaire_locatie_lengtegraad,primaire_locatie_breedtegraad
0,A-ALL-IM19087680NLD_1,2019-08-28 11:34:31,2020-09-09 14:32:53,vehicle_obstruction,brokenDownVehicle,4.346407,52.04192
1,A-ALL-IM19087680NLD_1,2019-08-28 11:34:31,2020-09-09 14:32:53,vehicle_obstruction,brokenDownVehicle,4.346407,52.04192
2,A-ALL-IM19087680NLD_1,2019-08-28 11:34:31,2020-09-09 14:32:53,vehicle_obstruction,brokenDownVehicle,4.346407,52.04192
3,A-ALL-IM19087680NLD_1,2019-08-28 11:34:31,2019-12-16 18:45:27,vehicle_obstruction,brokenDownVehicle,4.346407,52.04192
4,A-ALL-IM19087680NLD_1,2019-08-28 11:34:31,2020-09-09 14:32:53,vehicle_obstruction,brokenDownVehicle,4.346407,52.04192


In [171]:
# Unique IDs
unique_id_count = incidents_df['id'].nunique()
print(f'Unique IDs: {unique_id_count}')

# Convert coordinates to floats
incidents_df['primaire_locatie_lengtegraad'] = pd.to_numeric(incidents_df['primaire_locatie_lengtegraad'], errors='coerce')
incidents_df['primaire_locatie_breedtegraad'] = pd.to_numeric(incidents_df['primaire_locatie_breedtegraad'], errors='coerce')

# Drop rows with NaN values in 'primaire_locatie_lengtegraad' or 'primaire_locatie_breedtegraad'
incidents_df.dropna(subset=['primaire_locatie_lengtegraad', 'primaire_locatie_breedtegraad'], inplace=True)

print('Rows:', len(incidents_df))

Unique IDs: 39304
Rows: 161962


C:\Users\migue\AppData\Local\Temp\ipykernel_29764\388227697.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incidents_df['primaire_locatie_lengtegraad'] = pd.to_numeric(incidents_df['primaire_locatie_lengtegraad'], errors='coerce')
C:\Users\migue\AppData\Local\Temp\ipykernel_29764\388227697.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incidents_df['primaire_locatie_breedtegraad'] = pd.to_numeric(incidents_df['primaire_locatie_breedtegraad'], errors='coerce')
C:\Users\migue\AppData\Local\Temp\ipyk

Filter incidents by region, road direction and lanes

In [172]:
# Filter incidents_df for entries within the bounding box of Ring Rotterdam
incidents_df = incidents_df[
    (incidents_df['primaire_locatie_breedtegraad'] >= min_latitude) & 
    (incidents_df['primaire_locatie_breedtegraad'] <= max_latitude) & 
    (incidents_df['primaire_locatie_lengtegraad'] >= min_longitude) & 
    (incidents_df['primaire_locatie_lengtegraad'] <= max_longitude)
]
print('Rows:', len(incidents_df))

# Convert 'starttijd' and 'eindtijd' to datetime
incidents_df['starttijd'] = pd.to_datetime(incidents_df['starttijd'])
incidents_df['eindtijd'] = pd.to_datetime(incidents_df['eindtijd'])

# Define your date range
start_date = pd.Timestamp('2020-03-01')
end_date = pd.Timestamp('2020-05-31')

# Filter the DataFrame to only include incidents within the specified date range
incidents_df = incidents_df[(incidents_df['starttijd'] >= start_date) & (incidents_df['eindtijd'] <= end_date)]
print('Rows:', len(incidents_df))

incidents_df = incidents_df.drop_duplicates(subset=['id', 'starttijd', 'eindtijd'])
print('Rows:', len(incidents_df))
incidents_df = incidents_df.drop_duplicates(subset='id', keep='last')
print('Rows:', len(incidents_df))
incidents_df = incidents_df.drop_duplicates(subset=['primaire_locatie_breedtegraad', 'primaire_locatie_lengtegraad'], keep='first')
print('Rows:', len(incidents_df))


# After filtering within the boundary box, this is another list of coordinates to exclude
exclude_points = [
    (51.922989, 4.471057), (51.918350, 4.484596), (51.922234, 4.496322), (51.911800, 4.463905), (51.900166, 4.468047), (51.895542, 4.494501),
    (51.895172, 4.531459), (51.875618, 4.510518), (51.861649, 4.431097), (51.885815, 4.580630), (51.885921, 4.580412), (51.885960, 4.580658),
    (51.885960, 4.580583), (51.883751, 4.580418), (51.883572, 4.580498), (51.883202, 4.580448), (51.880310, 4.579200), (51.879910, 4.578885),
    (51.945419, 4.530178), (51.945461, 4.530137), (51.945221, 4.5317659), (51.944641, 4.532023), (51.945221, 4.531765), (51.950294, 4.530092),
    (51.952148, 4.534003), (51.952148, 4.534270), (51.952080, 4.534332), (51.950920, 4.529327), (51.950989, 4.528609), (51.940910, 4.485485), 
    (51.940910, 4.485485), (51.941509, 4.485172), (51.941509, 4.485007), (51.941669, 4.484843), (51.949543, 4.353939), (51.948170, 4.354400),
    (51.947891, 4.354633), (51.947689, 4.355468), (51.946350, 4.355833), (51.946320, 4.355840), (51.946209, 4.355927), (51.946266, 4.356325),
    (51.945320, 4.356800), (51.945229, 4.357145), (51.945011, 4.357248), (51.944839, 4.357110), (51.944740, 4.357085), (51.943821, 4.357853), 
    (51.940529, 4.360508), (51.935448, 4.363258), (51.933723, 4.364162), (51.931461, 4.364682), (51.930069, 4.365018), (51.927929, 4.364900),
    (51.927750, 4.364575), (51.923660, 4.373622), (51.923691, 4.373650), (51.926979, 4.375842), (51.926964, 4.374980), (51.925201, 4.367567),
    (51.952171, 4.530847), (51.875278, 4.354077), (51.875591, 4.354473), (51.875576, 4.355828), (51.875679, 4.357201), (51.876099, 4.357139),
    (51.875980, 4.357467), (51.875481, 4.359515), (51.875500, 4.360422), (51.876122, 4.360257), (51.876011, 4.357473), (51.876289, 4.364535), 
    (51.875854, 4.364359), (51.875870, 4.364915), (51.876083, 4.365851), (51.875942, 4.365623), (51.875790, 4.365723), (51.875648, 4.366027), 
    (51.876381, 4.367132), (51.876099, 4.370345), (51.875874, 4.370180), (51.875591, 4.370445), (51.875851, 4.370840), (51.875900, 4.370735), 
    (51.875980, 4.371172), (51.876152, 4.371324), (51.875969, 4.371545), (51.875710, 4.372345), (51.875019, 4.372492), (51.874519, 4.374115), 
    (51.875801, 4.374798), (51.875881, 4.375285), (51.875381, 4.375182), (51.874733, 4.375641), (51.874264, 4.375369), (51.874111, 4.375957), 
    (51.874420, 4.377025), (51.874538, 4.378173), (51.874840, 4.378160), (51.875648, 4.366023), (51.876019, 4.367993), (51.875889, 4.370208),
    (51.875809, 4.374853), (51.877060, 4.394310), (51.866180, 4.554630), (51.866161, 4.554617), (51.866161, 4.554617), (51.866379, 4.553540), 
    (51.866390, 4.553525), (51.866470, 4.553150), (51.866508, 4.553443), (51.866501, 4.553165), (51.866840, 4.553167), (51.867249, 4.553345), 
    (51.868992, 4.552337), (51.868889, 4.552237), (51.868191, 4.553162), (51.868771, 4.551740), (51.868591, 4.551348), (51.868401, 4.550557), 
    (51.868130, 4.550400), (51.868061, 4.550508), (51.867889, 4.550237), (51.867699, 4.550232), (51.867611, 4.550183), (51.866329, 4.554102), 
    (51.866428, 4.551800), (51.878231, 4.578028), (51.876942, 4.575860), (51.876659, 4.577937), (51.876308, 4.577647), (51.876942, 4.575860), 
    (51.899059, 4.549817), (51.898319, 4.549875), (51.914955, 4.536726), (51.914181, 4.536775), (51.913639, 4.535338), (51.914200, 4.534308), 
    (51.919033, 4.533980), (51.919369, 4.534172), (51.919540, 4.534540), (51.919941, 4.534437), (51.925449, 4.539258), (51.925671, 4.539332), 
    (51.925766, 4.539145), (51.926350, 4.536135), (51.946259, 4.534313), (51.946678, 4.534058), (51.946716, 4.532471), (51.950241, 4.530770), 
    (51.949131, 4.528152), (51.949581, 4.527830), (51.949421, 4.527205), (51.949291, 4.531802), (51.949268, 4.531682), (51.950294, 4.528594), 
    (51.950447, 4.528435), (51.952862, 4.543250), (51.953800, 4.550055), (51.953800, 4.550110), (51.953819, 4.550505), (51.953831, 4.550555), 
    (51.952244, 4.538253), (51.941460, 4.485115), (51.942299, 4.485537), (51.937962, 4.461262), (51.938049, 4.460963), (51.938187, 4.460633), 
    (51.938389, 4.462012), (51.938599, 4.461818), (51.938629, 4.462075), (51.938652, 4.461869), (51.938450, 4.461135), (51.939018, 4.461298), 
    (51.938751, 4.460382), (51.938438, 4.460071), (51.935230, 4.450202), (51.934990, 4.450263), (51.930771, 4.439460), (51.930779, 4.438747), 
    (51.931129, 4.438678), (51.931412, 4.439023), (51.931389, 4.440078), (51.931469, 4.439567), (51.931461, 4.439450), (51.931843, 4.437848), 
    (51.931759, 4.437483), (51.936253, 4.432872), (51.937592, 4.430878), (51.937611, 4.430793), (51.938061, 4.430650), (51.938580, 4.430238), 
    (51.939461, 4.429335), (51.939541, 4.429075), (51.940361, 4.428165), (51.943359, 4.424520), (51.943447, 4.424546), (51.943668, 4.423973), 
    (51.945332, 4.422122), (51.945602, 4.422104), (51.945911, 4.421798), (51.946430, 4.421225), (51.946579, 4.420773), (51.946850, 4.420798), 
    (51.947231, 4.420264), (51.947929, 4.419143), (51.948708, 4.418588), (51.949261, 4.418088), (51.950039, 4.417750), (51.952354, 4.414354),
    (51.951427, 4.414880), (51.952339, 4.421722), (51.952148, 4.421567), (51.950588, 4.415008), (51.950741, 4.415627), (51.950600, 4.415953),
    (51.949940, 4.415242), (51.949924, 4.415112), (51.949699, 4.415435), (51.950840, 4.416965), (51.950539, 4.417313), (51.950569, 4.417713),
    
]

# New points to exclude, converted to tuple format directly
excluded_points_direction_1 = [
    (51.923900604248, 4.35466814041138), (51.9241104125977, 4.35554218292236), (51.9246292114258, 4.35763502120972),
    (51.9247512817383, 4.35827779769897), (51.9247398376465, 4.36068201065063), (51.9242782592773, 4.36249685287476),
    (51.9252014160156, 4.36000490188599), (51.9255599975586, 4.36229801177979), (51.9251594543457, 4.36224985122681), 
    (51.9249687194824, 4.36242485046387), (51.9252014160156, 4.36000490188599), (51.9255599975586, 4.36229801177979), 
    (51.9251594543457, 4.36224985122681), (51.9249687194824, 4.36242485046387), (51.9258193969727, 4.36309003829956), 
    (51.9266700744629, 4.36446809768677), (51.9256591796875, 4.36515998840332), (51.9250602722168, 4.3653450012207), 
    (51.9253311157227, 4.36619520187378), (51.924373626709, 4.36554479598999), (51.9234809875488, 4.36615991592407), 
    (51.9238510131836, 4.36739301681519), (51.9271087646484, 4.36429309844971), (51.9237518310547, 4.36356782913208), 
    (51.9261817932129, 4.36565017700195), (51.9261589050293, 4.36578702926636), (51.9267387390137, 4.36571502685547), 
    (51.9269409179688, 4.36741018295288), (51.9268112182617, 4.36824178695679), (51.9266815185547, 4.36839008331299), 
    (51.9266204833984, 4.36851787567139), (51.9257698059082, 4.36871290206909), (51.9243392944336, 4.36829996109009), 
    (51.9242401123047, 4.36826705932617), (51.923168182373,   4.36705493927002), (51.9286499023438, 4.36554193496704), 
    (51.9287414550781, 4.3652172088623), (51.9289207458496, 4.36533498764038), (51.9258003234863, 4.37132501602173), 
    (51.9259605407715, 4.37219715118408), (51.9252891540527, 4.37242794036865), (51.9251594543457, 4.37256193161011), 
    (51.9259796142578, 4.373694896698), (51.9261589050293, 4.37411689758301), (51.926139831543, 4.37755298614502),  
    (51.9261283874512, 4.37786197662354),  (51.9259986877441, 4.37787199020386), (51.9263000488281, 4.3822078704834),  
    (51.9264793395996, 4.38349008560181),  (51.9263916015625, 4.40309286117554), (51.9259223937988, 4.40365409851074),  
    (51.9263916015625, 4.40309286117554),  (51.9271202087402, 4.4028000831604), (51.9272003173828, 4.40343713760376),  
    (51.9271583557129, 4.40386199951172),  (51.927188873291, 4.40419006347656), (51.9272918701172, 4.40386295318604),
    (51.9252090454102, 4.40360689163208),  (51.9251899719238, 4.40391492843628), (51.9251861572266, 4.40408420562744), (51.9243507385254, 4.40822505950928), 
    (51.92333984375, 4.40959978103638),  (51.9228172302246, 4.41238689422607),  (51.9229316711426, 4.41253280639648),  (51.9228401184082, 4.41293287277222), 
    (51.9227905273438, 4.41310501098633),  (51.9229011535645, 4.41577816009521),  (51.9228782653809, 4.41712999343872),  (51.9237785339355, 4.42068719863892), 
    (51.9241104125977, 4.4204797744751),  (51.9244232177734, 4.42163801193237),  (51.924430847168, 4.42134714126587),  (51.9246711730957, 4.42186689376831), 
    (51.9246711730957, 4.42187976837158),  (51.9248809814453, 4.4234619140625),  (51.9257392883301, 4.42452478408813),  (51.9257507324219, 4.42463207244873), 
    (51.9257583618164, 4.42471218109131), (51.9259910583496, 4.42563486099243),  (51.9262619018555, 4.42587995529175), (51.92626953125, 4.42606210708618), 
    (51.9277381896973, 4.42979288101196), (51.9279098510742, 4.43019008636475),  (51.9286117553711, 4.43189477920532), 
    (51.9289207458496, 4.43263673782349), (51.9296607971191, 4.43430709838867), (51.9299011230469, 4.4350471496582), 
    (51.9297332763672, 4.43525695800781), (51.9297409057617, 4.43537187576294), (51.9299697875977, 4.43591499328613), (51.9307289123535, 4.43780279159546), 
    (51.9317092895508, 4.44009399414063), (51.9322509765625, 4.4373631477356), (51.9325485229492, 4.43696689605713), (51.9329605102539, 4.43752813339233), 
    (51.9338912963867, 4.43650722503662), (51.9338912963867, 4.43560695648193), (51.9348220825195, 4.43448400497437), (51.9346885681152, 4.43462514877319),
    (51.9242706298828, 4.41823196411133), (51.9239654541016, 4.42031908035278), (51.9257507324219, 4.42461776733398), (51.9306411743164, 4.43768692016602),
    (51.932430267334, 4.44171714782715), (51.9324111938477, 4.44175815582275), (51.9326705932617, 4.44222497940063), (51.9329299926758, 4.44237184524536),
    (51.9333992004395, 4.4433331489563), (51.9335403442383, 4.44355487823486), (51.9334182739258, 4.4434609413147), (51.9338607788086, 4.44469976425171),
    (51.9344100952148, 4.44643211364746), (51.9345932006836, 4.4474139213562), (51.934928894043, 4.44853019714355), (51.9356803894043, 4.45319795608521),
    (51.9363899230957, 4.45513486862183), (51.9367294311523, 4.45716190338135), (51.936939239502, 4.45684289932251), (51.9369888305664, 4.45673322677612),
    (51.9370002746582, 4.45670509338379), (51.9372787475586, 4.45758199691772), (51.9390182495117, 4.46229314804077), (51.9390983581543, 4.46253681182861),
    (51.9391403198242, 4.46358013153076), (51.9396591186523, 4.46399021148682), (51.939640045166, 4.46425485610962), (51.9397201538086, 4.46432685852051),
    (51.9398918151855, 4.4657940864563), (51.9404296875, 4.46973705291748), (51.9407691955566, 4.47173500061035), (51.9407997131348, 4.4720778465271),
    (51.9408416748047, 4.47267007827759), (51.9408302307129, 4.47284460067749), (51.9410133361816, 4.47429370880127), (51.9411087036133, 4.47438287734985),
    (51.9411201477051, 4.47453498840332), (51.9411315917969, 4.47457695007324), (51.9415588378906, 4.47814702987671), (51.9416809082031, 4.47915983200073),
    (51.9416694641113, 4.47998714447021), (51.9417381286621, 4.48034191131592), (51.9419784545898, 4.48208713531494), (51.941951751709, 4.48280191421509),
    (51.9419097900391, 4.48282814025879), (51.9418601989746, 4.48300981521606), (51.942211151123, 4.48478698730469), (51.9422912597656, 4.48648977279663),
    (51.9424095153809, 4.4868049621582), (51.9424781799316, 4.48730278015137), (51.9425392150879, 4.48857927322388), (51.942699432373, 4.48957014083862),
    (51.9441108703613, 4.4924898147583), (51.9446792602539, 4.49404001235962), (51.9451789855957, 4.49626207351685), (51.945728302002, 4.5018630027771),
    (51.9457092285156, 4.50085687637329), (51.9457206726074, 4.50076484680176), (51.9459495544434, 4.5026650428772), (51.9459495544434, 4.50342178344727),
    (51.9459495544434, 4.50368309020996), (51.9461097717285, 4.5045371055603), (51.9462013244629, 4.50551795959473), (51.9463691711426, 4.5066351890564),
    (51.9464111328125, 4.50761699676514), (51.9466590881348, 4.50885486602783), (51.9467010498047, 4.5091028213501), (51.9467315673828, 4.50950813293457),
    (51.9329795837402, 4.4375147819519),  (51.933422088623, 4.4434609413147),   (51.9392204284668, 4.46291780471802), (51.9440689086914, 4.49256181716919),
    (51.9457206726074, 4.50082492828369), (51.9477806091309, 4.51455307006836), (51.947639465332, 4.51448678970337),  (51.9472961425781, 4.51325798034668),
    (51.9478607177734, 4.51601934432983), (51.9483299255371, 4.51730489730835), (51.9485092163086, 4.51826190948486), (51.9482192993164, 4.51892995834351),
    (51.9486541748047, 4.51861476898193), (51.94873046875, 4.51880693435669),   (51.9488105773926, 4.51936817169189), (51.9492301940918, 4.52071189880371),
    (51.9496803283691, 4.5237021446228),  (51.9502105712891, 4.5245680809021),  (51.950309753418, 4.52491283416748),  (51.9507102966309, 4.52661180496216),
    (51.9507904052734, 4.52667999267578), (51.9508209228516, 4.52714204788208), (51.9508094787598, 4.52737808227539), (51.9513893127441, 4.5336799621582),
    (51.9515609741211, 4.53380823135376), (51.950611114502, 4.53309011459351),  (51.9504508972168, 4.53266286849976), (51.9520301818848, 4.53985214233398),
    (51.9520797729492, 4.53997707366943), (51.9521484375, 4.54083013534546),    (51.9525909423828, 4.54407978057861), (51.952579498291, 4.54411315917969),
    (51.9528617858887, 4.54599809646606), (51.9536361694336, 4.54963254928589), (51.9244689941406, 4.41835021972656), (51.9514007568359, 4.52768278121948), 
    (51.9502487182617, 4.53247976303101), (51.9501495361328, 4.5323281288147), (51.9501609802246, 4.5323281288147),  (51.9469108581543, 4.53063011169434), (51.946949005127, 4.52997779846191),  (51.9468002319336, 4.52978706359863),
    (51.9461708068848, 4.53250217437744), (51.9460601806641, 4.53250789642334), (51.9436416625977, 4.53246688842773), (51.9439697265625, 4.53295803070068),
    (51.943229675293, 4.53259992599487),  (51.942741394043, 4.53364515304565),  (51.9426498413086, 4.53373193740845), (51.9417991638184, 4.53371000289917),
    (51.9418983459473, 4.53394794464111), (51.9415817260742, 4.53408813476563), (51.9415016174316, 4.53408813476563), (51.9410400390625, 4.53400707244873),
    (51.9408302307129, 4.53414487838745), (51.9400100708008, 4.53465986251831), (51.9388198852539, 4.5350399017334),  (51.9387397766113, 4.5352201461792),
    (51.9386215209961, 4.53522491455078), (51.9384498596191, 4.53533220291138), (51.9379692077637, 4.53537750244141), (51.9346313476563, 4.53690719604492),
    (51.9345397949219, 4.53693723678589), (51.9337120056152, 4.53714561462402), (51.9333381652832, 4.53745985031128), (51.932689666748, 4.53773498535156),
    (51.9322700500488, 4.53787517547607), (51.9319915771484, 4.53778648376465), (51.9316711425781, 4.53791189193726), (51.9316787719727, 4.53800678253174),
    (51.9309005737305, 4.53816986083984), (51.9304809570313, 4.53830718994141), (51.9305992126465, 4.53811693191528), (51.9300804138184, 4.53855991363525),
    (51.9294013977051, 4.53864669799805), (51.9296188354492, 4.53896522521973), (51.9275207519531, 4.53873682022095), (51.9263000488281, 4.53628587722778),
    (51.9266738891602, 4.53858470916748), (51.92626953125, 4.5385217666626),   (51.9247817993164, 4.53789520263672), (51.9458122253418, 4.53252792358398), (51.9427490234375, 4.53367519378662),
    (51.9408302307129, 4.5341329574585),  (51.932689666748,   4.53773307800293), (51.9322814941406, 4.53786516189575), (51.9323501586914,  4.5378680229187),(51.9327011108398, 4.53773212432861),
    (51.924072265625, 4.53741502761841),   (51.9240341186523, 4.53765535354614), (51.923698425293, 4.53751182556152),   (51.9231796264648, 4.53697299957275),
    (51.922550201416, 4.53645801544189),   (51.9217681884766, 4.53599977493286), (51.9213600158691, 4.53577709197998),  (51.9212799072266, 4.53568696975708),
    (51.9213104248047, 4.53563690185547),  (51.9205894470215, 4.53542804718018), (51.9203796386719, 4.53524494171143),  (51.9190101623535, 4.53505706787109),
    (51.9186592102051, 4.53473520278931),  (51.9184799194336, 4.53473711013794), (51.9172019958496, 4.53468322753906),  (51.9169311523438, 4.53478288650513),
    (51.9165382385254, 4.53484487533569),  (51.9163246154785, 4.53485774993896), (51.9160385131836, 4.53505992889404),  (51.9156875610352, 4.53594589233398),
    (51.9155616760254, 4.53524684906006),  (51.9155197143555, 4.53528785705566), (51.9154510498047, 4.53523015975952),  (51.9154014587402, 4.53528022766113),
    (51.9153900146484, 4.53533315658569),  (51.9152297973633, 4.53536319732666), (51.9129104614258, 4.53675508499146),  (51.9131088256836, 4.5368971824646),
    (51.9122200012207, 4.53779792785645),  (51.9122009277344, 4.53772306442261), (51.9119682312012, 4.53757476806641),  (51.9121170043945, 4.53728199005127),
    (51.9120101928711, 4.53717517852783),  (51.912281036377,  4.53699493408203), (51.9125099182129, 4.53608798980713),  (51.9112777709961, 4.53799104690552),
    (51.9088134765625, 4.53953981399536),  (51.9082183837891, 4.53987503051758), (51.9063301086426, 4.54130887985229),  (51.9051399230957, 4.54219198226929),
    (51.9046936035156, 4.54248476028442),  (51.9035797119141, 4.54351186752319), (51.9030570983887, 4.5436429977417),   (51.9024391174316, 4.54416179656982),
    (51.9022331237793, 4.54436779022217),  (51.9021301269531, 4.54453992843628), (51.901741027832,  4.54484176635742),  (51.9014205932617, 4.54479885101318),
    (51.9007186889648, 4.54531192779541),  (51.8998184204102, 4.54623508453369), (51.8994598388672, 4.54633188247681),  (51.8995513916016, 4.54665279388428),
    (51.8991813659668, 4.54684782028198),  (51.8991088867188, 4.5469822883606), (51.8989410400391, 4.54724979400635),  (51.8985595703125, 4.54687976837158),
    (51.898811340332,  4.54764795303345),  (51.8984336853027, 4.54793214797974), (51.8982582092285, 4.54800987243652),  (51.8982315063477, 4.54870986938477),
    (51.8982391357422, 4.54646682739258),  (51.8981132507324, 4.54661083221436), (51.9238815307617, 4.53786420822144),    (51.9199485778809, 4.53501987457275),    (51.9154014587402, 4.53530693054199),    (51.9130592346191, 4.53650188446045),
    (51.902271270752, 4.54433488845825),    (51.8977432250977, 4.54660892486572),    (51.8978500366211, 4.54937696456909),    (51.8970413208008, 4.5488429069519),
    (51.8975410461426, 4.54864692687988),    (51.8964729309082, 4.54687023162842),    (51.8974494934082, 4.55026721954346),    (51.8973350524902, 4.55020809173584),
    (51.8974227905273, 4.5503945350647),    (51.8973007202148, 4.55082988739014),    (51.8970909118652, 4.55148983001709),    (51.8969497680664, 4.55169010162354),
    (51.8967399597168, 4.55252695083618),    (51.8965110778809, 4.55278778076172),    (51.8964195251465, 4.553542137146),    (51.896369934082, 4.55374002456665),
    (51.8963890075684, 4.55224514007568),    (51.8962097167969, 4.55356788635254),    (51.8958015441895, 4.55316305160522),    (51.8959884643555, 4.55452823638916),
    (51.895809173584, 4.55472278594971),    (51.8955993652344, 4.55548286437988),    (51.8953094482422, 4.55459213256836),    (51.8954200744629, 4.55444002151489),
    (51.8953704833984, 4.55605983734131),    (51.8950881958008, 4.55633783340454),    (51.8945388793945, 4.55741500854492),    (51.894474029541, 4.55764627456665),
    (51.8942413330078, 4.55825185775757),    (51.8941192626953, 4.55841302871704),    (51.8941612243652, 4.55870199203491),    (51.8939590454102, 4.55884790420532),
    (51.8934211730957, 4.55919313430786),    (51.8934516906738, 4.55978012084961),    (51.8933792114258, 4.55976486206055),    (51.8931198120117, 4.5600700378418),
    (51.8935852050781, 4.56051969528198),    (51.8935012817383, 4.56059789657593),    (51.8922691345215, 4.56068181991577),    (51.8921394348145, 4.56085681915283),
    (51.8916206359863, 4.56112003326416),    (51.8916893005371, 4.56147193908691),    (51.8914794921875, 4.56142282485962),    (51.8916854858398, 4.56161212921143),
    (51.8908081054688, 4.5617470741272),    (51.8904914855957, 4.56196784973145),    (51.8901710510254, 4.56241321563721),    (51.8898582458496, 4.56232213973999),
    (51.8886795043945, 4.56270790100098),    (51.8888397216797, 4.56294298171997),    (51.8877716064453, 4.56297016143799),    (51.8869781494141, 4.56372690200806),
    (51.8866882324219, 4.56380701065063),    (51.886661529541, 4.56386518478394),    (51.8860282897949, 4.56372880935669),    (51.8856391906738, 4.56408214569092),
    (51.8850898742676, 4.56418704986572),    (51.8846015930176, 4.5644850730896),    (51.8843994140625, 4.56438302993774),    (51.8843994140625, 4.56450700759888),
    (51.884391784668, 4.56484317779541),    (51.8842697143555, 4.5648832321167),    (51.8835601806641, 4.56487512588501),    (51.8837203979492, 4.56504678726196),
    (51.8831787109375, 4.56503677368164),    (51.8829307556152, 4.56438493728638),    (51.8827209472656, 4.56495809555054),    (51.8822288513184, 4.56548023223877),
    (51.8817596435547, 4.56554508209229),    (51.8807792663574, 4.56595468521118),    (51.8807487487793, 4.56607294082642),    (51.8786010742188, 4.56840515136719),
    (51.8779106140137, 4.56839990615845),    (51.8769798278809, 4.57027006149292),    (51.8765182495117, 4.57223176956177),    (51.8765106201172, 4.56966781616211),
    (51.8756713867188, 4.570237159729),    (51.8760986328125, 4.57490205764771),    (51.8752059936523, 4.57515907287598),    (51.8751182556152, 4.57389688491821),
    (51.8749809265137, 4.5739631652832),    (51.8751106262207, 4.57315683364868),    (51.8755302429199, 4.57647323608398),    (51.8756904602051, 4.57669305801392),
    (51.8749504089355, 4.57751178741455),    (51.8749504089355, 4.57092523574829), (51.8975410461426, 4.54864978790283), (51.8958511352539, 4.54650020599365), (51.8962631225586, 4.54811000823975), (51.896240234375, 4.55058193206787),
    (51.8975410461426, 4.54864978790283), (51.8973999023438, 4.55020523071289), (51.8960113525391, 4.55723476409912), (51.8961486816406, 4.55755519866943),
    (51.8961296081543, 4.55766677856445), (51.8954887390137, 4.5589280128479),  (51.8944244384766, 4.56002283096313), (51.8916893005371, 4.56163597106934),
    (51.8801498413086, 4.56785488128662), (51.8798217773438, 4.56594181060791), (51.8803596496582, 4.56546497344971), (51.8769416809082, 4.568922996521),
    (51.8746681213379, 4.57081699371338), (51.8746337890625, 4.57094764709473), (51.8740615844727, 4.56945896148682), (51.8743934631348, 4.572181224823),
    (51.9143104553223, 4.53519201278687), (51.8957595825195, 4.55513715744019), (51.923038482666, 4.36568307876587),  (51.9227485656738, 4.36579322814941),
    (51.9221305847168, 4.36552286148071), (51.9219093322754, 4.36568784713745), (51.9218292236328, 4.36566686630249), (51.9214897155762, 4.36575508117676),
    (51.9212799072266, 4.36577320098877), (51.919750213623, 4.3659029006958),  (51.9196090698242, 4.36597681045532), (51.9191703796387, 4.36596822738647),
    (51.9188575744629, 4.36623334884644), (51.9179763793945, 4.36641407012939), (51.9152297973633, 4.36731815338135), (51.9151496887207, 4.36743021011353),
    (51.914421081543, 4.36756801605225), (51.9134712219238, 4.36775207519531), (51.9130783081055, 4.36789178848267), (51.9128189086914, 4.36802196502686),
    (51.9119834899902, 4.36881494522095), (51.9119682312012, 4.36856985092163), (51.911018371582, 4.36888122558594), (51.9112892150879, 4.36881017684937),
    (51.910099029541, 4.36896800994873), (51.9101181030273, 4.36948013305664), (51.9099197387695, 4.36970520019531), (51.9104118347168, 4.37109279632568),
    (51.9106101989746, 4.37174320220947), (51.909740447998, 4.36823797225952), (51.9099197387695, 4.36970520019531), (51.9095916748047, 4.36987018585205),
    (51.9093856811523, 4.37014245986938), (51.909351348877, 4.37272691726685), (51.9087600708008, 4.37023210525513), (51.9092597961426, 4.36795997619629),
    (51.9093818664551, 4.36805200576782), (51.9082183837891, 4.37009716033936), (51.9079399108887, 4.37023210525513), (51.9079818725586, 4.3701319694519),
    (51.9079818725586, 4.37006187438965), (51.9065818786621, 4.37065315246582), (51.9055480957031, 4.37048721313477), (51.9050216674805, 4.36994695663452),
    (51.9026298522949, 4.36978816986084), (51.9024887084961, 4.36977815628052), (51.9023208618164, 4.36988019943237), (51.9013137817383, 4.36991357803345),
    (51.8953399658203, 4.37448787689209), (51.8953704833984, 4.3760027885437),  (51.8948707580566, 4.37492179870605), (51.8947982788086, 4.37510776519775),
    (51.8947715759277, 4.37519311904907), (51.8946800231934, 4.37533187866211), (51.8945503234863, 4.37515211105347), (51.8942604064941, 4.37619781494141),
    (51.8939208984375, 4.37608003616333), (51.892219543457, 4.37766790390015),  (51.8902397155762, 4.37792301177979), (51.889949798584, 4.3783278465271),
    (51.8897018432617, 4.37842512130737), (51.8897705078125, 4.3780632019043),  (51.8881607055664, 4.37898778915405), (51.886547088623, 4.37979364395142),
    (51.8852615356445, 4.38030290603638), (51.8842391967773, 4.3808422088623),  (51.8841896057129, 4.38115215301514), (51.8836097717285, 4.3812050819397),
    (51.8831481933594, 4.38164758682251), (51.882740020752, 4.38175010681152),  (51.8824996948242, 4.38194513320923), (51.8823089599609, 4.38197183609009),
    (51.8788795471191, 4.38400602340698), (51.8782196044922, 4.38416719436646),  (51.8775100708008, 4.38476181030273), (51.8776588439941, 4.3864278793335),
    (51.8759613037109, 4.38306283950806), (51.8762016296387, 4.38342094421387),  (51.8757209777832, 4.38261699676514), (51.8752899169922, 4.38152313232422),
    (51.9084815979004, 4.37038803100586), (51.9080581665039, 4.37013006210327), (51.9080085754395, 4.37007808685303), (51.9024887084961, 4.36979722976685),
    (51.9020309448242, 4.36979722976685), (51.8762283325195, 4.38347721099854), (51.8762702941895, 4.38350820541382), (51.8762435913086, 4.38347196578979),
    (51.8727951049805, 4.38067483901978), (51.8722496032715, 4.38195514678955), (51.8722991943359, 4.38246011734009), (51.8728713989258, 4.38505220413208),
    (51.8718414306641, 4.3917818069458),  (51.8722801208496, 4.3889799118042),  (51.8727188110352, 4.38781213760376), (51.8729705810547, 4.38974189758301),
    (51.8730812072754, 4.38697290420532), (51.8731307983398, 4.38633823394775), (51.8734436035156, 4.38679027557373), (51.8743515014648, 4.38751220703125),
    (51.9143104553223, 4.53519201278687), (51.8957595825195, 4.55513715744019), (51.923038482666, 4.36568307876587), (51.9227485656738, 4.36579322814941),
    (51.9221305847168, 4.36552286148071), (51.9219093322754, 4.36568784713745), (51.9218292236328, 4.36566686630249), (51.9214897155762, 4.36575508117676),
    (51.9212799072266, 4.36577320098877), (51.919750213623, 4.3659029006958), (51.9196090698242, 4.36597681045532), (51.9191703796387, 4.36596822738647),
    (51.9188575744629, 4.36623334884644), (51.9179763793945, 4.36641407012939), (51.9152297973633, 4.36731815338135), (51.9151496887207, 4.36743021011353),
    (51.914421081543, 4.36756801605225), (51.9134712219238, 4.36775207519531), (51.9130783081055, 4.36789178848267), (51.9128189086914, 4.36802196502686),
    (51.9119834899902, 4.36881494522095), (51.9119682312012, 4.36856985092163), (51.911018371582, 4.36888122558594), (51.9112892150879, 4.36881017684937),
    (51.910099029541, 4.36896800994873), (51.9101181030273, 4.36948013305664), (51.9099197387695, 4.36970520019531), (51.9104118347168, 4.37109279632568),
    (51.9106101989746, 4.37174320220947), (51.909740447998, 4.36823797225952), (51.9099197387695, 4.36970520019531), (51.9095916748047, 4.36987018585205),
    (51.9093856811523, 4.37014245986938), (51.909351348877, 4.37272691726685), (51.9087600708008, 4.37023210525513), (51.9092597961426, 4.36795997619629),
    (51.9093818664551, 4.36805200576782), (51.9082183837891, 4.37009716033936), (51.9079399108887, 4.37023210525513), (51.9079818725586, 4.3701319694519),
    (51.9079818725586, 4.37006187438965), (51.9065818786621, 4.37065315246582), (51.9055480957031, 4.37048721313477), (51.9050216674805, 4.36994695663452),
    (51.9026298522949, 4.36978816986084), (51.9024887084961, 4.36977815628052), (51.9023208618164, 4.36988019943237), (51.9013137817383, 4.36991357803345),
    (51.8953399658203, 4.37448787689209), (51.8953704833984, 4.3760027885437), (51.8948707580566, 4.37492179870605), (51.8947982788086, 4.37510776519775),
    (51.8947715759277, 4.37519311904907), (51.8946800231934, 4.37533187866211), (51.8945503234863, 4.37515211105347), (51.8942604064941, 4.37619781494141),
    (51.8939208984375, 4.37608003616333), (51.892219543457, 4.37766790390015), (51.8902397155762, 4.37792301177979), (51.889949798584, 4.3783278465271),
    (51.8897018432617, 4.37842512130737), (51.8897705078125, 4.3780632019043), (51.8881607055664, 4.37898778915405), (51.886547088623, 4.37979364395142),
    (51.8852615356445, 4.38030290603638), (51.8842391967773, 4.3808422088623), (51.8841896057129, 4.38115215301514), (51.8836097717285, 4.3812050819397),
    (51.8831481933594, 4.38164758682251), (51.882740020752, 4.38175010681152), (51.8824996948242, 4.38194513320923), (51.8823089599609, 4.38197183609009),
    (51.8788795471191, 4.38400602340698), (51.8782196044922, 4.38416719436646), (51.8775100708008, 4.38476181030273), (51.8776588439941, 4.3864278793335),
    (51.8759613037109, 4.38306283950806), (51.8762016296387, 4.38342094421387), (51.8757209777832, 4.38261699676514), (51.8752899169922, 4.38152313232422),
    (51.909610748291, 4.36791086196899),   (51.8735961914063, 4.38389587402344), (51.8733177185059, 4.38408708572388), (51.876049041748, 4.38757991790771),
    (51.8749046325684, 4.38989067077637),  (51.8722381591797, 4.39225816726685),  (51.872200012207, 4.39245986938477),  (51.872859954834, 4.39535713195801),
    (51.8727607727051, 4.39542818069458),  (51.8725891113281, 4.39530801773071),  (51.872631072998, 4.39563322067261),  (51.8734703063965, 4.40420007705688),
    (51.8733596801758, 4.40635776519775),  (51.8732681274414, 4.41005706787109),  (51.8733329772949, 4.41018629074097), (51.8730926513672, 4.41304206848145),
    (51.8728294372559, 4.41512680053711),  (51.8721008300781, 4.41966485977173),  (51.8723793029785, 4.41904306411743), (51.8722496032715, 4.41955995559692),
    (51.8722534179688, 4.42014312744141),  (51.8720283508301, 4.42154216766357),  (51.8720207214355, 4.42169713973999), (51.8716697692871, 4.42379999160767),
    (51.8710594177246, 4.42454481124878),  (51.8712310791016, 4.42484188079834),  (51.870979309082, 4.42626190185547),  (51.8712844848633, 4.42573547363281),
    (51.8704681396484, 4.42722797393799),  (51.8704109191895, 4.42866516113281),  (51.8706893920898, 4.42786693572998), (51.8700904846191, 4.42922687530518),
    (51.8701095581055, 4.43112277984619),  (51.8699607849121, 4.43185997009277),  (51.8699417114258, 4.43208312988281), (51.8694801330566, 4.43307209014893),
    (51.8690147399902, 4.43362617492676),  (51.8692893981934, 4.4340648651123),   (51.869270324707, 4.43485021591187),  (51.8692474365234, 4.43533849716187),
    (51.8687286376953, 4.43668508529663),  (51.8684310913086, 4.43739700317383),  (51.8671607971191, 4.44103193283081), (51.8671913146973, 4.4411768913269),
    (51.8667411804199, 4.4423131942749),   (51.8666191101074, 4.44251203536987),  (51.8675994873047, 4.44149494171143), (51.867618560791, 4.44208192825317),
    (51.8668556213379, 4.44450426101685),  (51.8667106628418, 4.44500207901001),  (51.866340637207, 4.44580411911011),  (51.8661117553711, 4.44785785675049),
    (51.8656387329102, 4.44971513748169),  (51.8655662536621, 4.45007467269897),  (51.8655395507813, 4.45026302337646), (51.8651390075684, 4.45091819763184),
    (51.8650436401367, 4.45121383666992),  (51.8645095825195, 4.45365810394287),  (51.8642883300781, 4.45468807220459), (51.863899230957, 4.45477294921875),
    (51.8638458251953, 4.45670461654663),  (51.8634910583496, 4.45788288116455),  (51.8624801635742, 4.4591817855835),   (51.8623085021973, 4.45987701416016),
    (51.8622398376465, 4.46022176742554),  (51.8619918823242, 4.46110010147095),  (51.8619003295898, 4.46136522293091), (51.8626289367676, 4.46219682693481),
    (51.8616409301758, 4.46437501907349),  (51.8613700866699, 4.46543979644775),  (51.861499786377, 4.46719312667847),   (51.8618087768555, 4.46876811981201),
    (51.8617706298828, 4.46957302093506),  (51.8616485595703, 4.47232294082642),  (51.8618812561035, 4.4729151725769),   (51.8618507385254, 4.47363805770874),
    (51.8619918823242, 4.47600698471069),  (51.8621406555176, 4.48027992248535),  (51.8624000549316, 4.48153305053711), (51.8624382019043, 4.4819278717041),
    (51.8625183105469, 4.48281192779541),  (51.8626518249512, 4.48243188858032),  (51.8627395629883, 4.48327779769897), (51.8627815246582, 4.48344278335571),
    (51.8633575439453, 4.4865870475769),   (51.8633117675781, 4.4882230758667),   (51.8633499145508, 4.48847007751465), (51.8637733459473, 4.48941230773926),
    (51.8641090393066, 4.49229001998901),  (51.8641548156738, 4.49373531341553),  (51.864372253418, 4.49364757537842),  (51.8643417358398, 4.49436187744141),
    (51.8643989562988, 4.49512147903442),  (51.864330291748, 4.4954719543457),    (51.8646850585938, 4.49649858474731), (51.8648109436035, 4.49878978729248),
    (51.8647003173828, 4.49983978271484),  (51.8649711608887, 4.50060796737671),  (51.8650932312012, 4.50079774856567), (51.8648300170898, 4.50106287002563),
    (51.8651390075684, 4.50179004669189),  (51.8652038574219, 4.50223541259766),  (51.8652305603027, 4.5040078163147),   (51.8651885986328, 4.50839281082153),
    (51.8655395507813, 4.50799608230591),  (51.8655776977539, 4.50935125350952),  (51.865119934082, 4.51094913482666),   (51.865535736084, 4.51229000091553),
    (51.8654289245605, 4.51442813873291),  (51.8649406433105, 4.51927995681763),  (51.8648490905762, 4.52047777175903), (51.864429473877, 4.5221471786499),
    (51.8642501831055, 4.52556180953979),  (51.8641891479492, 4.52599811553955),  (51.8640403747559, 4.52723789215088), (51.8640098571777, 4.52750635147095),
    (51.8638191223145, 4.52842807769775),  (51.8638305664063, 4.52882814407349),  (51.8637504577637, 4.52902984619141), (51.8637313842773, 4.52928686141968),
    (51.8636894226074, 4.52985715866089),  (51.8636016845703, 4.53036499023438),  (51.8636436462402, 4.53111028671265), (51.863525390625, 4.53383350372314),
    (51.8634605407715, 4.53410005569458),  (51.8731002807617, 4.3920521736145), (51.87353515625, 4.39818334579468),  (51.8735694885254, 4.399658203125),     (51.8736000061035, 4.40259599685669),
    (51.871280670166, 4.42474699020386),  (51.8705101013184, 4.42718982696533), (51.870849609375, 4.42784214019775),   (51.8707427978516, 4.42850351333618),
    (51.8711395263672, 4.42899322509766), (51.8685111999512, 4.43746185302734), (51.8682632446289, 4.43939399719238),  (51.8681297302246, 4.44321298599243),
    (51.8681716918945, 4.44349193572998), (51.8673553466797, 4.44650745391846), (51.8672790527344, 4.44670820236206),  (51.8651504516602, 4.45124816894531),
    (51.8651695251465, 4.45142793655396), (51.8635139465332, 4.45804834365845), (51.8614196777344, 4.46547794342041),  (51.8627166748047, 4.48235607147217),
    (51.8633193969727, 4.48817777633667), (51.8634796142578, 4.48827981948853), (51.8644790649414, 4.49545192718506),  (51.8647499084473, 4.49784994125366),
    (51.8655853271484, 4.51529026031494), (51.8659591674805, 4.51545763015747), (51.8685989379883, 4.51576995849609),  (51.8683280944824, 4.51595497131348),
    (51.8676414489746, 4.51668214797974), (51.8638916015625, 4.51486206054688), (51.8634300231934, 4.51476716995239),  (51.8626747131348, 4.51507091522217),
    (51.8620986938477, 4.51651000976563), (51.8634376525879, 4.51815891265869), (51.8613891601563, 4.51498317718506),  (51.8611907958984, 4.51601696014404),
    (51.8604888916016, 4.51632785797119), (51.8596115112305, 4.51577186584473), (51.8588714599609, 4.51580286026001),  (51.8634376525879, 4.51815891265869),
    (51.8711586, 4.4290652),  (51.8692017, 4.4379549),  (51.8653107, 4.4511981),  (51.8647499, 4.4978099),
    (51.8652191, 4.5173302),  (51.8650627, 4.5195427),  (51.8648109, 4.5214849),  (51.8650627, 4.5195427),
    (51.8648109, 4.5214849),  (51.8637390, 4.5294180),  (51.8634987, 4.5361919),  (51.8635902, 4.5367651),
    (51.8637085, 4.5372920),  (51.8636665, 4.5380125),  (51.8635902, 4.5386772),  (51.8636894, 4.5389051),
    (51.8639946, 4.5408745),  (51.8639717, 4.5413451),  (51.8642159, 4.5422201),  (51.8643303, 4.5430970),
    (51.8651237, 4.5461230),  (51.8651200, 4.5463929),  (51.8651009, 4.5471120),  (51.8660469, 4.5490384),
    (51.8669472, 4.5515432),  (51.8674698, 4.5536299),  (51.8680992, 4.5552778),  (51.8601685, 4.5159178), (51.865421295166, 4.51661491394043),  (51.8652877807617, 4.52360820770264),  (51.8649101257324, 4.52496290206909),  (51.8637886047363, 4.52944421768188),
    (51.8635902404785, 4.53859281539917),  (51.8636817932129, 4.53874778747559),  (51.8639907836914, 4.5414080619812),   (51.8656196594238, 4.54777002334595),
    (51.8661651611328, 4.55054616928101),  (51.8675117492676, 4.55111312866211),  (51.8683280944824, 4.55073213577271),  (51.8676681518555, 4.55068016052246),
    (51.8683013916016, 4.55530977249146),  (51.8682518005371, 4.55577993392944),  (51.8683204650879, 4.55592203140259),  (51.8688430786133, 4.55640888214111),
    (51.8688507080078, 4.55801820755005),  (51.8691368103027, 4.55777788162231),  (51.8691596984863, 4.55800676345825),  (51.8698997497559, 4.5593581199646),
    (51.869987487793, 4.5621395111084),    (51.8702507019043, 4.56259679794312),  (51.8703308105469, 4.5629267692565),
    (51.8735694885254, 4.39358282089233),  (51.8735504150391, 4.39243316650391),  (51.8737716674805, 4.39828491210938),  (51.873950958252, 4.39973783493042),
    (51.8736305236816, 4.41217708587646),  (51.8658714294434, 4.54801797866821),  (51.8679618835449, 4.55043983459473),  (51.8675498962402, 4.5509729385376),
    (51.8675308227539, 4.5512318611145),   (51.8680686950684, 4.55401515960693),  (51.868350982666, 4.55515336990356),   (51.8683090209961, 4.55535507202148),
    (51.8684692382813, 4.55581521987915),  (51.8683891296387, 4.55599498748779),  (51.8712387084961, 4.56413793563843),  (51.871150970459, 4.56606483459473),
    (51.8717041015625, 4.5638108253479),   (51.8720016479492, 4.56480693817139),  (51.8718185424805, 4.56731700897217),  (51.8722991943359, 4.56632995605469),
    (51.871940612793, 4.56787776947021),   (51.8721389770508, 4.56839179992676),  (51.8723602294922, 4.56850814819336),  (51.8725395202637, 4.56944513320923),
    (51.872013092041, 4.56950426101685),   (51.8718795776367, 4.56955718994141),  (51.8714790344238, 4.57114791870117),  (51.8712501525879, 4.57179021835327),
    (51.8711204528809, 4.57383298873901),  (51.8718414306641, 4.57415819168091),  (51.8764495849609, 4.57415199279785),  (51.8731002807617, 4.57226514816284),
    (51.8727378845215, 4.57214307785034),  (51.8757019042969, 4.57669305801392),  (51.8751907348633, 4.57728815078735),  (51.8727569580078, 4.575364112854),
    (51.8722305297852, 4.57662773132324),  (51.8724517822266, 4.57662677764893),  (51.871280670166, 4.57679176330566),  (51.8710556030273, 4.5759334564209),
    (51.8709716796875, 4.57573318481445),  (51.871940612793, 4.57483816146851), 
    (51.8733444213867, 4.39602994918823),  (51.8681488037109, 4.44329309463501),  (51.8677406311035, 4.55058479309082),  (51.8668785095215, 4.54987192153931),
    (51.8671112060547, 4.55098295211792),  (51.8711013793945, 4.56068181991577),  (51.8717193603516, 4.563796043396  ),  (51.8757705688477, 4.5766429901123 ),
    (51.8752670288086, 4.57723617553711),  (51.8755493164063, 4.56877183914185),  (51.8713989257813, 4.57676792144775),  (51.8717308044434, 4.57673692703247),
    (51.8716888427734, 4.5764799118042 ),  (51.871940612793 , 4.57618808746338),  (51.8706398010254, 4.57340717315674),  (51.8702583312988, 4.57470512390137),
    (51.8703002929688, 4.57653522491455),  (51.8705902099609, 4.57689189910889),  (51.8698043823242, 4.57802724838257),  (51.8698616027832, 4.57853698730469),
    (51.8693618774414, 4.57607221603394),  (51.8689498901367, 4.57726192474365),  (51.8686103820801, 4.57748985290527),  (51.868278503418 , 4.57832193374634),
    (51.8674392700195, 4.581383228302  ),  (51.8667602539063, 4.58196687698364),  (51.8666801452637, 4.58213520050049),  (51.8599586486816, 4.59158992767334),
    (51.860969543457 , 4.59125804901123),  (51.8707313537598, 4.5731987953186),  (51.8702697753906, 4.57474517822266),  (51.8689918518066, 4.57653474807739),  (51.8692092895508, 4.57906484603882),
    (51.8720092773438, 4.57623291015625),  (51.8667144775391, 4.58339977264404),  (51.8664283752441, 4.58476305007935),  (51.8658218383789, 4.58523321151733),
    (51.8655586242676, 4.58460283279419),  (51.865291595459, 4.58472585678101),  (51.8650703430176, 4.58491182327271),  (51.8653297424316, 4.58501815795898),
    (51.8651885986328, 4.58522701263428),  (51.8654975891113, 4.58551597595215),  (51.8650093078613, 4.5859580039978),  (51.8655700683594, 4.58588695526123),
    (51.864990234375, 4.58727788925171),  (51.8645896911621, 4.58731508255005),  (51.8635101318359, 4.58828020095825),  (51.8636322021484, 4.58876085281372),
    (51.8638610839844, 4.58915185928345),  (51.863208770752, 4.5881872177124),  (51.8631629943848, 4.58840894699097),  (51.86328125, 4.58967018127441),
    (51.8630790710449, 4.589928150177),  (51.86328125, 4.59009981155396),  (51.8621978759766, 4.59045791625977),  (51.8623695373535, 4.59085559844971),
    (51.8619995117188, 4.59159803390503),  (51.8617897033691, 4.59201002120972),  (51.8616218566895, 4.59211015701294),  (51.861629486084, 4.5904278755188),
    (51.8617782592773, 4.59041309356689),  (51.8618583679199, 4.59024000167847),  (51.861930847168, 4.59049797058105),
]

excluded_points_same_direction_2 = [
    (51.8704261779785, 4.5746169090271), (51.8720397949219, 4.57605648040771), (51.8618011474609, 4.5904221534729), (51.8646774291992, 4.58578824996948),
    (51.8751907348633, 4.57000494003296), (51.8758850097656, 4.56937122344971), (51.8759689331055, 4.56919193267822), (51.8761787414551, 4.56905794143677),
    (51.8766708374023, 4.56855201721191), (51.8782196044922, 4.56739711761475), (51.8792381286621, 4.56659317016602), (51.8824691772461, 4.56486320495605),
    (51.8826904296875, 4.5648398399353), (51.8961906433105, 4.55285501480103), (51.8968696594238, 4.55081701278687), (51.8971481323242, 4.55019187927246),
    (51.8987503051758, 4.5471568107605), (51.899169921875, 4.54652309417725), (51.9000205993652, 4.54563283920288), (51.9084205627441, 4.53954982757568),
    (51.9087562561035, 4.53933525085449), (51.9102592468262, 4.53822994232178), (51.9300003051758, 4.53814697265625), (51.9295501708984, 4.53840208053589),
    (51.9272003173828, 4.53833818435669), (51.9230003356934, 4.53659009933472), (51.9220809936523, 4.53584718704224), (51.9216117858887, 4.53554487228394),
    (51.9183502197266, 4.53432512283325), (51.917839050293, 4.53433799743652), (51.9168891906738, 4.53452301025391), (51.9163208007813, 4.53471517562866),
    (51.9153709411621, 4.53504180908203), (51.9151802062988, 4.53514194488525), (51.9144592285156, 4.53555297851563), (51.9142112731934, 4.53569221496582),
    (51.911750793457, 4.53722476959229), (51.9121398925781, 4.53701782226563), (51.9102592468262, 4.53822994232178), (51.9087562561035, 4.53933525085449),
    (51.9084205627441, 4.53954982757568), (51.9000205993652, 4.54563283920288), (51.899169921875, 4.54652309417725), (51.8987503051758, 4.5471568107605),
    (51.8971481323242, 4.55019187927246), (51.8968696594238, 4.55081701278687), (51.8961906433105, 4.55285501480103), (51.8971481323242, 4.55019187927246),
    (51.9323883056641, 4.53744983673096), (51.9225921630859, 4.36619234085083), (51.9224395751953, 4.36625289916992), (51.910758972168, 4.36993217468262),
    (51.9113502502441, 4.37008285522461), (51.9077796936035, 4.37099981307983), (51.9066352844238, 4.37097930908203), (51.9058647155762, 4.37095260620117),
    (51.9034805297852, 4.37067985534668), (51.9029006958008, 4.37058782577515), (51.8960113525391, 4.37523698806763), (51.8951454162598, 4.37589740753174),
    (51.8943710327148, 4.37697792053223), (51.892650604248, 4.37781476974487), (51.8897018432617, 4.37889814376831), (51.8753700256348, 4.38906478881836),
    (51.8755912780762, 4.38871479034424), (51.8762702941895, 4.38789510726929), (51.8762893676758, 4.38788223266602), (51.8760414123535, 4.38801288604736),
    (51.873851776123, 4.40117502212524), (51.8738632202148, 4.40252685546875), (51.924088, 4.356499), (51.924339, 4.357895) 
]

excluded_points_same_direction_2_lower_part_ring = [
    (51.8732833862305, 4.41309642791748), (51.8731002807617, 4.41490697860718), 
    (51.8737411499023, 4.40293502807617), (51.8732490539551, 4.39184284210205), 
    (51.8740081787109, 4.39429187774658), (51.8738975524902, 4.39690208435059), 
    (51.8717956542969, 4.42599630355835), (51.8714256286621, 4.42581272125244), 
    (51.8701591491699, 4.43307685852051), (51.8684158325195, 4.43947839736938), 
    (51.8683090209961, 4.44017314910889), (51.8679428100586, 4.44073104858398), 
    (51.8678398132324, 4.44211721420288), (51.8679885864258, 4.44153499603271), 
    (51.867790222168, 4.44341278076172), (51.8668785095215, 4.44590187072754), 
    (51.8650131225586, 4.45278215408325), (51.8636436462402, 4.45817136764526), 
    (51.8630409240723, 4.46087121963501), (51.8621101379395, 4.46794605255127), 
    (51.862190246582, 4.46815299987793), (51.8623199462891, 4.47625494003296), 
    (51.8627052307129, 4.48076629638672), (51.8643569946289, 4.49218606948853), 
    (51.8647499084473, 4.49513292312622), (51.8650207519531, 4.49789381027222), 
    (51.8644905090332, 4.49272203445435), (51.8656005859375, 4.50515985488892), 
    (51.8657989501953, 4.51004219055176), (51.8657493591309, 4.51310777664185), 
    (51.8652534484863, 4.51961660385132), (51.8652000427246, 4.52090978622437), 
    (51.8650932312012, 4.5210280418396), (51.8649215698242, 4.52272176742554), 
    (51.8648986816406, 4.52288484573364), (51.8645172119141, 4.52533292770386), 
    (51.864143371582, 4.52961206436157), (51.8638801574707, 4.53628015518188), 
    (51.8638191223145, 4.53664016723633), (51.8638381958008, 4.5377368927002), 
    (51.867919921875, 4.55350017547607), (51.8680725097656, 4.55383205413818), 
    (51.8695983886719, 4.55737018585205), (51.8699607849121, 4.55835676193237),
    (51.8718414306641, 4.42590808868408), (51.8722991943359, 4.42254018783569),
    (51.869213104248, 4.43688201904297), (51.8653907775879, 4.45260810852051),
    (51.8657150268555, 4.50801086425781), (51.8661193847656, 4.54768991470337),
    (51.8662490844727, 4.54799509048462), (51.8706512451172, 4.55847692489624),
    (51.8732414245605, 4.56479215621948), (51.8735046386719, 4.56560087203979),
    (51.8736381530762, 4.56570720672607), (51.8737411499023, 4.56583499908447),
    (51.8652534484863, 4.45295000076294),   (51.8660202026367, 4.54737997055054), 
    (51.8706588745117, 4.55849695205688),  (51.8732795715332, 4.56481218338013), 
    (51.8741188049316, 4.5653247833252),   (51.861847, 4.590365)
]

excluded_points_after_mapping_incidents_with_cameras = [
    (51.938889, 4.534435), (51.934410, 4.536597), (51.941959, 4.533120), (51.943260, 4.532570),
    (51.936050, 4.535583), (51.935211, 4.535993), (51.935921, 4.535680), (51.933861, 4.536615),
    (51.932041, 4.537330), (51.922604, 4.366552), (51.922680, 4.366635), (51.875381, 4.566281)
]

# Combine both lists
total_excluded_points = exclude_points + excluded_points_direction_1  + excluded_points_same_direction_2 + excluded_points_same_direction_2_lower_part_ring + excluded_points_after_mapping_incidents_with_cameras
# Select the rows where mask is false
mask = incidents_df.apply(
    lambda row: any(
        abs(row['primaire_locatie_breedtegraad'] - point[0]) < 1e-6 and 
        abs(row['primaire_locatie_lengtegraad'] - point[1]) < 1e-6 for point in total_excluded_points
    ), axis=1
)

# Filter out the rows to exclude
incidents_df = incidents_df[~mask]
incidents_df = incidents_df.sort_values(by='starttijd')
incidents_df.reset_index(drop=True, inplace=True)
# Unique IDs
unique_id_count = incidents_df['id'].nunique()
print(f'Unique IDs: {unique_id_count}')

incidents_df = incidents_df.drop('eindtijd', axis=1)
incidents_df.tail()


Rows: 24595
Rows: 2395
Rows: 2267
Rows: 2237
Rows: 1462
Unique IDs: 266


,id,starttijd,type,gedetailleerd_type,primaire_locatie_lengtegraad,primaire_locatie_breedtegraad
261,RWS02_0000188391_188391,2020-05-29 12:57:36,vehicle_obstruction,emergencyVehicle,4.426357,51.871399
262,RWS02_0000188407_188407,2020-05-29 13:50:36,vehicle_obstruction,emergencyVehicle,4.560625,51.891628
263,RWS02_0000188661_188661,2020-05-29 20:27:35,vehicle_obstruction,emergencyVehicle,4.371268,51.908241
264,NLRWS_NLSIT002761414_1,2020-05-30 17:58:00,general_obstruction,objectOnTheRoad,4.367238,51.917259
265,RWS02_0000188891_188891,2020-05-30 19:41:36,vehicle_obstruction,emergencyVehicle,4.415205,51.922501


Define a Segment column to classify the incidents by section of the road for a better mapping to cameras

In [173]:
# Define the corner points of the rectangle
north_west = (51.925170, 4.361240)
north_east = (51.953850, 4.552810) 
south_east = (51.874718, 4.565618)
south_west = (51.875290, 4.388910)

def label_segments_lat_long_band(df, north_west, north_east, south_east, south_west):
    
    # Initialize the segment column
    df['segment'] = 'other'  # default value
    
    west_east_ids = [
    'RWS02_0000180197_180197',
    'RWS02_0000176238_176238',
    'RWS02_0000170485_170485',
    'RWS02_0000187304_187304',
    'RWS02_0000188891_188891',
    'NLRWS_NLSIT002749427_1',
    'RWS02_0000187703_187703',
    'RWS02_0000170367_170367',
    'RWS02_0000163115_163115'
]

    south_ids = [
        'RWS02_0000179510_179510',
        'RWS02_0000170357_170357',
        'RWS02_0000160018_160018',
        'RWS02_0000179862_179862',
        'RWS02_0000187160_187160',
        'RWS02_0000181493_181493',
        'RWS02_0000168238_168238',
        'RWS02_0000188501_188501',
        'RWS02_0000182711_182711',
        'RWS02_0000180044_180044',
        'RWS02_0000188096_188096',
        'RWS02_0000180765_180765',
        'RWS02_0000185491_185491',
        'RWS02_0000158152_158152',
        'RWS02_0000173747_173747'
        
    ]
    # Check each point against the latitude and longitude bands
    for i, row in df.iterrows():
        lat = row['primaire_locatie_breedtegraad']
        long = row['primaire_locatie_lengtegraad']
        id = row['id']
        
        if id in west_east_ids:
            df.at[i, 'segment'] = 'west-east'
        elif id in south_ids:
            df.at[i, 'segment'] = 'south'
        elif north_west[0] <= lat <= north_east[0]:
            df.at[i, 'segment'] = 'west-east'
        elif south_west[0] <= lat <= north_west[0] and north_west[1] <= long<= south_west[1]:
            df.at[i, 'segment'] = 'north'
        elif south_east[0] <= lat <= north_east[0] and 4.525670 <= long <= south_east[1]:
            df.at[i, 'segment'] = 'south'
        elif 51.923790 <= lat <= 51.924990  and 4.420440 <= long <= 4.406130:
            df.at[i, 'segment'] = 'west-east'
        elif min_latitude <= south_west[0]:
            df.at[i, 'segment'] = 'east-west'
    
    return df

incidents_df = label_segments_lat_long_band(incidents_df, north_west, north_east, south_east, south_west)
incidents_df.head()

,id,starttijd,type,gedetailleerd_type,primaire_locatie_lengtegraad,primaire_locatie_breedtegraad,segment
0,RWS02_0000157188_157188,2020-03-01 08:38:28,vehicle_obstruction,emergencyVehicle,4.366420,51.920952,north
1,RWS02_0000157197_157197,2020-03-01 10:00:57,vehicle_obstruction,emergencyVehicle,4.432840,51.928864,west-east
2,RWS02_0000157269_157269,2020-03-01 13:21:58,vehicle_obstruction,emergencyVehicle,4.456025,51.864422,east-west
3,RWS03_816372_1,2020-03-01 15:19:57,vehicle_obstruction,brokenDownVehicle,4.538568,51.909523,south
4,RWS02_0000157393_157393,2020-03-01 23:03:29,vehicle_obstruction,emergencyVehicle,4.439620,51.868599,east-west


Map generation of the incidents

In [175]:
# Calculate the average location for the center of the map
average_latitude = incidents_df['primaire_locatie_breedtegraad'].mean()
average_longitude = incidents_df['primaire_locatie_lengtegraad'].mean()

# Create a map centered around this average location
incidents_map = folium.Map(location=[average_latitude, average_longitude], zoom_start=12)

# Add markers for each incident
for idx, row in incidents_df.iterrows():
    folium.Marker(
        location=[row['primaire_locatie_breedtegraad'], row['primaire_locatie_lengtegraad']],
        icon=folium.Icon(color=color_for_segment(row['segment'])),
        popup=(
            f"ID: {row['id']}<br>"
            f"Start: {row['starttijd']}<br>"
            f"Type: {row['type']}<br>"
            f"Detail: {row['gedetailleerd_type']}<br>"
            f"Segment: {row['segment']}<br>"
            f"Latitude: {row['primaire_locatie_breedtegraad']:.6f}<br>"
            f"Longitude: {row['primaire_locatie_lengtegraad']:.6f}"
        ),
        tooltip='Click for more info'
    ).add_to(incidents_map)

# Save the map or display it in a Jupyter notebook
incidents_map.save('incidents_map.html')
incidents_map

Functions to convert latitudes and longitudes to mteric distances and to find the nearest cameras of an incident before and after it happened

In [176]:
def haversine(lon1, lat1, lon2, lat2):
    # Convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371  # Radius of earth in kilometers
    return c * r

def find_nearest_cameras(df_cameras, df_incidents, threshold_before_km, threshold_after_km):
    df_incidents['before'] = None
    df_incidents['after'] = None
    
    for i, incident_row in df_incidents.iterrows():
        segment = incident_row['segment']
        incident_lat = incident_row['primaire_locatie_breedtegraad']
        incident_lon = incident_row['primaire_locatie_lengtegraad']
        segment_cameras = df_cameras[df_cameras['segment'] == segment].copy()
        
        segment_cameras['distance_to_incident'] = segment_cameras.apply(
            lambda row: haversine(incident_lon, incident_lat, row['start_locatie_longitude'], row['start_locatie_latitude']),
            axis=1
        )
        
        # Dictionaries for 'before' and 'after' cameras
        before_cameras = {}
        after_cameras = {}
        
        for _, camera_row in segment_cameras.iterrows():
            camera_id = camera_row['id_meetlocatie']
            distance = camera_row['distance_to_incident']
            camera_lat = camera_row['start_locatie_latitude']
            camera_lon = camera_row['start_locatie_longitude']
            
            if segment == 'west-east':
                if camera_lon < incident_lon and distance <= threshold_before_km:
                    before_cameras[camera_id] = distance
                elif camera_lon >= incident_lon and distance <= threshold_after_km:
                    after_cameras[camera_id] = distance
            elif segment == 'south':
                if camera_lat > incident_lat and distance <= threshold_before_km:
                    before_cameras[camera_id] = distance
                elif camera_lat <= incident_lat and distance <= threshold_after_km:
                    after_cameras[camera_id] = distance
            elif segment == 'east-west':
                if camera_lon > incident_lon and distance <= threshold_before_km:
                    before_cameras[camera_id] = distance
                elif camera_lon <= incident_lon and distance <= threshold_after_km:
                    after_cameras[camera_id] = distance
            elif segment == 'north':
                if camera_lat < incident_lat and distance <= threshold_before_km:
                    before_cameras[camera_id] = distance
                elif camera_lat >= incident_lat and distance <= threshold_after_km:
                    after_cameras[camera_id] = distance
        
        # Sort the dictionaries by distance
        sorted_before_cameras = {k: v for k, v in sorted(before_cameras.items(), key=lambda item: item[1])}
        sorted_after_cameras = {k: v for k, v in sorted(after_cameras.items(), key=lambda item: item[1])}
        
        # Update the DataFrame with dictionaries
        df_incidents.at[i, 'before'] = sorted_before_cameras
        df_incidents.at[i, 'after'] = sorted_after_cameras

    return df_incidents

threshold_before_km = 2
threshold_after_km = 5

incidents_df = find_nearest_cameras(unique_rows, incidents_df, threshold_before_km, threshold_after_km)
incidents_df.tail() 

,id,starttijd,type,gedetailleerd_type,primaire_locatie_lengtegraad,primaire_locatie_breedtegraad,segment,before,after
261,RWS02_0000188391_188391,2020-05-29 12:57:36,vehicle_obstruction,emergencyVehicle,4.426357,51.871399,east-west,{'RWS01_MONIBAS_0150vwy0533ra': 0.039353705268...,{'RWS01_MONIBAS_0150vwy0531ra': 0.191299376662...
262,RWS02_0000188407_188407,2020-05-29 13:50:36,vehicle_obstruction,emergencyVehicle,4.560625,51.891628,south,{'RWS01_MONIBAS_0160vwx0219ra': 0.856010802632...,{'GEO0K_K_RWSTI357777': 1.9113097776268286}
263,RWS02_0000188661_188661,2020-05-29 20:27:35,vehicle_obstruction,emergencyVehicle,4.371268,51.908241,north,{'RWS01_MONICA_10D0040B403D68200005': 0.021037...,{'RWS01_MONIBAS_0040vwn0717ra': 0.273561803239...
264,NLRWS_NLSIT002761414_1,2020-05-30 17:58:00,general_obstruction,objectOnTheRoad,4.367238,51.917259,north,{'RWS01_MONIBAS_0041hrl0710ra': 0.039582973811...,{'RWS01_MONIBAS_0041hrl0706ra': 0.391533076393...
265,RWS02_0000188891_188891,2020-05-30 19:41:36,vehicle_obstruction,emergencyVehicle,4.415205,51.922501,west-east,{'RWS01_MONIBAS_0201hrr0269ra': 0.176246494380...,{'RWS01_MONIBAS_0201hrr0272ra': 0.138248633355...


Functions to deal with special cases like the corners. Here to get the cameras neded (before or after) we look at the adjacent segment

In [177]:
def find_edge_case_cameras_before(df_cameras, df_incidents, edge_case_ids, threshold_before_km):
    for incident_id in edge_case_ids:
        incident_index = df_incidents[df_incidents['id'] == incident_id].index[0]
        incident = df_incidents.loc[incident_index]
        incident_lat = incident['primaire_locatie_breedtegraad']
        incident_lon = incident['primaire_locatie_lengtegraad']
        segment = incident['segment']

        # Determine the adjacent segment to look for 'before' cameras
        if segment == 'east-west':
            adjacent_segment = 'south'
        else:
            continue 
        
        # Filter cameras by the adjacent segment
        adjacent_cameras = df_cameras[df_cameras['segment'] == adjacent_segment].copy()
        
        # Compute distances to incident
        adjacent_cameras['distance_to_incident'] = adjacent_cameras.apply(
            lambda row: haversine(incident_lon, incident_lat, row['start_locatie_longitude'], row['start_locatie_latitude']),
            axis=1
        )

        # For 'east-west' segment incidents, find 'before' cameras to the right of the incident
        if segment == 'east-west':
            before_cameras = adjacent_cameras[adjacent_cameras['start_locatie_longitude'] > incident_lon]

        # Filter based on distance and sort
        before_cameras = before_cameras[before_cameras['distance_to_incident'] <= threshold_before_km]
        before_cameras = before_cameras.nsmallest(2, 'distance_to_incident')
        before_cameras_dict = {row['id_meetlocatie']: row['distance_to_incident'] for _, row in before_cameras.iterrows()}

        # Update the 'before' column for the edge case incidents in the main DataFrame
        df_incidents.at[incident_index, 'before'] = before_cameras_dict

def find_edge_case_cameras_after(df_cameras, df_incidents, edge_case_ids, threshold_after_km):
    for incident_id in edge_case_ids:
        incident_index = df_incidents[df_incidents['id'] == incident_id].index[0]
        incident = df_incidents.loc[incident_index]
        incident_lat = incident['primaire_locatie_breedtegraad']
        incident_lon = incident['primaire_locatie_lengtegraad']
        segment = incident['segment']

        # For 'north' segment incidents, check cameras above in the 'west-east' segment
        # For 'east-west' segment incidents, check cameras to the left in the 'north' segment
        if segment == 'north':
            adjacent_segment = 'west-east'
            adjacent_cameras = df_cameras[df_cameras['segment'] == adjacent_segment]
            after_cameras = adjacent_cameras[adjacent_cameras['start_locatie_latitude'] > incident_lat]
        elif segment == 'east-west':
            adjacent_segment = 'north'
            adjacent_cameras = df_cameras[df_cameras['segment'] == adjacent_segment]
            after_cameras = adjacent_cameras[adjacent_cameras['start_locatie_longitude'] < incident_lon]
        else:
            continue

        # Compute distances to incident
        after_cameras['distance_to_incident'] = after_cameras.apply(
            lambda row: haversine(incident_lon, incident_lat, row['start_locatie_longitude'], row['start_locatie_latitude']),
            axis=1
        )

        # Filter based on distance and sort
        after_cameras = after_cameras[after_cameras['distance_to_incident'] <= threshold_after_km]
        after_cameras = after_cameras.nsmallest(2, 'distance_to_incident')
        after_cameras_dict = {row['id_meetlocatie']: row['distance_to_incident'] for _, row in after_cameras.iterrows()}

        # Update the 'after' column for the edge case incidents in the main DataFrame
        df_incidents.at[incident_index, 'after'] = after_cameras_dict

edge_case_ids_east_west = [
    'RWS02_0000160698_160698', 'RWS02_0000180207_180207',
]
edge_case_ids_after = [
    'RWS02_0000171155_171155'
]


threshold_before_km = 2
threshold_after_km = 5
find_edge_case_cameras_before(unique_rows, incidents_df, edge_case_ids_east_west, threshold_before_km)
find_edge_case_cameras_after(unique_rows, incidents_df, edge_case_ids_after, threshold_after_km)


Map to show incident with before and after cameras in a range of 2 km before the incident and 5 km after the incident. The distances are also displayed

In [153]:
# Locate the incident by ID instead of index
incident_id = 'RWS02_0000157188_157188' 
incident = incidents_df[incidents_df['id'] == incident_id].iloc[0]  # Use .iloc[0] to get the first match
incident_lat = incident['primaire_locatie_breedtegraad']
incident_lon = incident['primaire_locatie_lengtegraad']

# Retrieve camera details from 'unique_rows' for the selected incident
before_cameras_dict = incident['before']
after_cameras_dict = incident['after']

# Convert the dictionaries to DataFrames
before_cameras_df = pd.DataFrame(list(before_cameras_dict.items()), columns=['id_meetlocatie', 'distance_to_incident'])
after_cameras_df = pd.DataFrame(list(after_cameras_dict.items()), columns=['id_meetlocatie', 'distance_to_incident'])

# Merge details from 'unique_rows' DataFrame
before_cameras_details = pd.merge(unique_rows, before_cameras_df, on='id_meetlocatie')
after_cameras_details = pd.merge(unique_rows, after_cameras_df, on='id_meetlocatie')

# Create a map centered around the incident location
incident_map = folium.Map(location=[incident_lat, incident_lon], zoom_start=13)

# Function to add cameras to the map
def add_cameras_to_map(cameras_df, map_object, icon_color):
    for idx, row in cameras_df.iterrows():
        folium.Marker(
            location=[row['start_locatie_latitude'], row['start_locatie_longitude']],
            popup=f'Camera ID: {row["id_meetlocatie"]}\nDistance: {row["distance_to_incident"]:.2f} km',
            icon=folium.Icon(color=icon_color)
        ).add_to(map_object)

# Add 'before' cameras to the map with red icons
add_cameras_to_map(before_cameras_details, incident_map, 'red')

# Add 'after' cameras to the map with green icons
add_cameras_to_map(after_cameras_details, incident_map, 'green')

# Add a marker for the incident location
folium.Marker(
    [incident_lat, incident_lon],
    popup='Incident Location',
    icon=folium.Icon(color='blue', icon='info-sign')
).add_to(incident_map)

# Save the map to an HTML file
incident_map.save('incidents_with_cameras_linked.html')

# Display the map in a Jupyter Notebook
incident_map

Interactive clickable map. This can be deleted afterwards

In [165]:
from ipyleaflet import Map, Marker, Popup, Icon
import ipywidgets as widgets
from IPython.display import display

# Central location for the map
center = (51.9225, 4.47917)

# Create the map
m = Map(center=center, zoom=12)

# Lists to store coordinates
camera_clicked_points = []
incident_clicked_points = []

# Output widget to display list of coordinates
output = widgets.Output()

def handle_camera_click(**kwargs):
    if kwargs.get('type') == 'click':
        coords = kwargs.get('coordinates')
        camera_clicked_points.append(coords)
        with output:
            print(f"Clicked on Camera: {coords}")
            print("All Camera points:", camera_clicked_points)

def handle_incident_click(**kwargs):
    if kwargs.get('type') == 'click':
        coords = kwargs.get('coordinates')
        incident_clicked_points.append(coords)
        with output:
            print(f"Clicked on Incident: {coords}")
            print("All Incident points:", incident_clicked_points)

# Add markers for cameras
for _, cam_row in unique_rows.iterrows():
    camera_marker = Marker(
        location=(cam_row['start_locatie_latitude'], cam_row['start_locatie_longitude']),
        draggable=False,
        icon=Icon(icon_url='https://leafletjs.com/examples/custom-icons/leaf-red.png', icon_size=[38, 95], icon_anchor=[22, 94], markerColor='blue'),
        title=f"Camera ID: {cam_row['id_meetlocatie']}"
    )
    camera_marker.on_click(handle_camera_click)
    m.add_layer(camera_marker)

# Add markers for incidents
for _, inc_row in incidents_df.iterrows():
    incident_marker = Marker(
        location=(inc_row['primaire_locatie_breedtegraad'], inc_row['primaire_locatie_lengtegraad']),
        draggable=False,
        icon=Icon(icon_color='red', icon_size=[25, 41], icon_anchor=[12, 41]),
        title=f"Incident ID: {inc_row['id']}"
    )
    incident_marker.on_click(handle_incident_click)
    m.add_layer(incident_marker)

# Display the map and the output widget
display(m, output)

# Save the map to an HTML file
m.save('interactive_map.html')


Map(center=[51.9225, 4.47917], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

Output()

In [162]:
print(incident_clicked_points)

[[51.8652534484863, 4.45295000076294], [51.8660202026367, 4.54737997055054], [51.8706588745117, 4.55849695205688], [51.8732795715332, 4.56481218338013], [51.8741188049316, 4.5653247833252]]


In [163]:
print(camera_clicked_points)

[]


In [183]:
driver='postgresql'
username='dab_ds23241a_223'
dbname=username # it is the same as the username
password='Bh6KFgcWazMMCPvZ'
server='bronto.ewi.utwente.nl'
port='5432'

# Creating the connetcion pool for SQL
engine = create_engine(f'{driver}://{username}:{password}@{server}:{port}/{dbname}')
incidents_df['before'] = incidents_df['before'].apply(json.dumps)
incidents_df['after'] = incidents_df['after'].apply(json.dumps)
incidents_df.to_sql('Incidents_With_Cameras', engine,schema='project', index=False, if_exists='append')

266